# 智能在线搜索平台

## 1. 导入依赖并初始化客户端

In [ ]:
! pip install --upgrade zhipuai
! pip install --upgrade google-auth google-auth-httplib2 google-auth-oauthlib google-api-python-client requests

In [1]:
import os
import openai
from openai import OpenAI
import glob
import shutil

import numpy as np
import pandas as pd

import json
import io
import inspect
import requests
import re
import random
import string

from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
import base64


from bs4 import BeautifulSoup
import dateutil.parser as parser
import tiktoken
from lxml import etree

## 初始化客户端
api_key = os.getenv("ZHIPU_API_KEY")


from zhipuai import ZhipuAI
client = ZhipuAI(api_key=api_key)

## 2. 大模型回答问题策略测试

In [9]:
response = client.chat.completions.create(
  model="glm-4",
  messages=[
    {"role": "system", "content": "根据用户输入的问题进行回答，如果知道问题的答案，请回答问题答案，如果不知道问题答案，请回复‘抱歉，这个问题我并不知道’"},
    {"role": "user", "content": "请问，什么是机器学习？"}
  ]
)
response.choices[0].message.content

'机器学习是人工智能的一个核心领域，它让计算机能够模拟人类的学习和思考方式。通过使用大量数据和算法，机器学习可以使计算机学会分类、回归和聚类等任务，从而让计算机能够从数据中提取知识，进行决策和预测。在机器学习的流程中，包括数据获取、特征工程、建立模型、模型评估以及调参等步骤。深度学习和强化学习是机器学习的两个重要分支。深度学习主要用于处理复杂结构数据的建模问题，而强化学习则让机器在探索环境中通过试错进行学习。'

In [21]:
response = client.chat.completions.create(
  model="glm-4",
  messages=[
    {"role": "system", "content": "根据用户输入的问题进行回答，如果知道问题的答案，请回答问题答案，如果不知道问题答案，请回复‘抱歉，这个问题我并不知道’"},
    {"role": "user", "content": "介绍一下关于GPT-6的猜想"}
  ]
)
response.choices[0].message.content

'抱歉，这个问题我并不知道。\n\n到2023为止，GPT-6尚未被公开提及或发布。GPT（Generative Pre-trained Transformer）系列模型由OpenAI开发，至今已发布了多个版本，如GPT-2和GPT-3。关于未来版本的GPT，如GPT-6，可能会有很多猜想和预期，但具体的内容和功能我无法提供，因为那将基于未来尚未公开的技术和信息。如果GPT-6在您提问之后有了新的消息，我可能无法获取那些最新的信息。'

增加外部函数测试一下，看大模型回答问题的时候是先去调用外部函数，还是先尝试自己回答。

In [2]:
def auto_functions(functions_list):
    """
    Chat模型的functions参数编写函数
    :param functions_list: 包含一个或者多个函数对象的列表；
    :return：满足Chat模型functions参数要求的functions对象
    """
    def functions_generate(functions_list):
        # 创建空列表，用于保存每个函数的描述字典
        functions = []
        # 对每个外部函数进行循环
        for function in functions_list:
            # 读取函数对象的函数说明
            function_description = inspect.getdoc(function)
            # 读取函数的函数名字符串
            function_name = function.__name__

            system_prompt = '以下是某的函数说明：%s,输出结果必须是一个JSON格式的字典，只输出这个字典即可，前后不需要任何前后修饰或说明的语句' % function_description
            user_prompt = '根据这个函数的函数说明，请帮我创建一个JSON格式的字典，这个字典有如下5点要求：\
                           1.字典总共有三个键值对；\
                           2.第一个键值对的Key是字符串name，value是该函数的名字：%s，也是字符串；\
                           3.第二个键值对的Key是字符串description，value是该函数的函数的功能说明，也是字符串；\
                           4.第三个键值对的Key是字符串parameters，value是一个JSON Schema对象，用于说明该函数的参数输入规范。\
                           5.输出结果必须是一个JSON格式的字典，只输出这个字典即可，前后不需要任何前后修饰或说明的语句' % function_name

            response = client.chat.completions.create(
                              model="glm-4",
                              messages=[
                                {"role": "system", "content": system_prompt},
                                {"role": "user", "content": user_prompt}
                              ]
                            )
            json_str=response.choices[0].message.content.replace("```json","").replace("```","")
            json_function_description=json.loads(json_str)
            json_str={"type": "function","function":json_function_description}
            functions.append(json_str)
        return functions
    ## 最大可以尝试4次
    max_attempts = 4
    attempts = 0

    while attempts < max_attempts:
        try:
            functions = functions_generate(functions_list)
            break  # 如果代码成功执行，跳出循环
        except Exception as e:
            attempts += 1  # 增加尝试次数
            print("发生错误：", e)
            if attempts == max_attempts:
                print("已达到最大尝试次数，程序终止。")
                raise  # 重新引发最后一个异常
            else:
                print("正在重新运行...")
    return functions

In [3]:
def run_conversation(messages, functions_list=None, model="glm-4"):
    """
    能够自动执行外部函数调用的对话模型
    :param messages: 必要参数，字典类型，输入到Chat模型的messages参数对象
    :param functions_list: 可选参数，默认为None，可以设置为包含全部外部函数的列表对象
    :param model: Chat模型，可选参数，默认模型为glm-4
    :return：Chat模型输出结果
    """
    # 如果没有外部函数库，则执行普通的对话任务
    if functions_list == None:
        response = client.chat.completions.create(
                        model=model,
                        messages=messages,
                        )
        response_message = response.choices[0].message
        final_response = response_message.content
        
    # 若存在外部函数库，则需要灵活选取外部函数并进行回答
    else:
        # 创建functions对象
        tools = auto_functions(functions_list)

        # 创建外部函数库字典
        available_functions = {func.__name__: func for func in functions_list}

        # 第一次调用大模型
        response = client.chat.completions.create(
                        model=model,
                        messages=messages,
                        tools=tools,
                        tool_choice="auto", )
        response_message = response.choices[0].message


        tool_calls = response_message.tool_calls

        if tool_calls:

            #messages.append(response_message)
            messages.append(response.choices[0].message.model_dump())
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)
                ## 真正执行外部函数的就是这儿的代码
                function_response = function_to_call(**function_args)
                messages.append(
                    {
                        "role": "tool",
                        "content": function_response,
                        "tool_call_id": tool_call.id,
                    }
                ) 
            ## 第二次调用模型
            second_response = client.chat.completions.create(
                model=model,
                messages=messages,
                tools=tools
            ) 
            # 获取最终结果
            print(second_response.choices[0].message)
            final_response = second_response.choices[0].message.content
        else:
            final_response = response_message.content
                
    return final_response

In [4]:
def ml_answer(q='什么是机器学习'):
    """
    解释什么是机器学习，返回机器学习的定义和解释
    :param q: 询问的问题，非必要参数，字符串类型对象
    :return：返回机器学习的定义和解释
    """
    return("机器学习是一种人工智能（AI）的分支领域，旨在使计算机系统通过学习和经验改进性能。")

In [25]:
ml_answer()

'机器学习是一种人工智能（AI）的分支领域，旨在使计算机系统通过学习和经验改进性能。'

In [5]:
functions_list = [ml_answer]

In [6]:
tools = auto_functions(functions_list)
tools

[{'type': 'function',
  'function': {'name': 'ml_answer',
   'description': '返回机器学习的定义和解释',
   'parameters': {'type': 'object',
    'properties': {'q': {'type': 'string',
      'description': '询问的问题，非必要参数，字符串类型对象'}},
    'required': [],
    'additionalProperties': False}}}]

In [28]:
response = client.chat.completions.create(
        model="glm-4",
        messages=[
            {"role": "user", "content": "解释什么是机器学习？"}
        ],
        tools=tools,
        tool_choice="auto"
    )
response.choices[0].message

CompletionMessage(content=None, role='assistant', tool_calls=[CompletionMessageToolCall(id='call_8578780768202319090', function=Function(arguments='{"q":"什么是机器学习？"}', name='ml_answer'), type='function')])

我们发现，加入了外部函数以后，大模型回答同一个问题，有时候去调外部函数，有时候是自己去回答的。
但是我们想要的效果是，问任何问题的时候，大模型先自己判断一下自己知不知道，如果知道，那就基于自己的知识进行回答
如果自己不知道的时候采取调用外部函数回答问题。

In [30]:
response = client.chat.completions.create(
        model="glm-4",
        messages=[
            {"role":"system","content": "对于用户提的问题，如果知道就直接回答，不知道才通过调用外部函数回答"},
            {"role": "user", "content": "什么是机器学习?"}
        ],
        tools=tools,
        tool_choice="auto"
    )
response.choices[0].message

CompletionMessage(content=None, role='assistant', tool_calls=[CompletionMessageToolCall(id='call_8578777950703544485', function=Function(arguments='{"q":"什么是机器学习?"}', name='ml_answer'), type='function')])

In [7]:
response = client.chat.completions.create(
        model="glm-4",
        messages=[
            {"role":"system","content": "对于用户提的问题，如果知道就直接回答，不知道才通过调用外部函数回答"},
            {"role": "user", "content": "什么是大模型?"}
        ],
        tools=tools,
        tool_choice="auto"
    )
response.choices[0].message

CompletionMessage(content=None, role='assistant', tool_calls=[CompletionMessageToolCall(id='call_8581405955293239233', function=Function(arguments='{"q":"什么是大模型"}', name='ml_answer'), type='function')])

发现提示词都起不到特别好的作用，不稳定。
所以后面需要解决这个问题

## 3. 谷歌搜索API开发

In [22]:
google_search_key = os.getenv("GOOGLE_SEARCH_API_KEY")
cse_id = os.getenv("CSE_ID")

In [23]:
search_term = "OpenAI"

In [24]:
import requests

# Step 1.构建请求
url = "https://www.googleapis.com/customsearch/v1"

# Step 2.设置查询参数（还有很多参数）
params = {
    'q': search_term,           # 搜索关键词
    'key': google_search_key,   # 谷歌搜索API Key
    'cx': cse_id                # CSE ID
}

# Step 3.发送GET请求
response = requests.get(url, params=params)

# Step 4.解析响应
data = response.json()

In [25]:
data

{'kind': 'customsearch#search',
 'url': {'type': 'application/json',
  'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'},
 'queries': {'request': [{'title': 'Google Custom Search - OpenAI',
    'totalResults': '204000000',
    'searchTerms': 'OpenAI',
    'count': 10,
    'startIndex': 1,
    'inputEncoding': 'utf8',
    'outputEncoding': 'utf8',
    'safe': 'off',
    'cx': 

In [26]:
data['items']

[{'kind': 'customsearch#result',
  'title': 'OpenAI',
  'htmlTitle': '<b>OpenAI</b>',
  'link': 'https://openai.com/',
  'displayLink': 'openai.com',
  'snippet': 'Products · Customblogcover. Introducing improvements to the fine-tuning API and expanding our custom models program · Noauth Cover. Start using ChatGPT\xa0...',
  'htmlSnippet': 'Products &middot; Customblogcover. Introducing improvements to the fine-tuning API and expanding our custom models program &middot; Noauth Cover. Start using ChatGPT&nbsp;...',
  'cacheId': 'ZJjhy9NmZvMJ',
  'formattedUrl': 'https://openai.com/',
  'htmlFormattedUrl': 'https://<b>openai</b>.com/',
  'pagemap': {'metatags': [{'og:image': 'https://images.openai.com/blob/fb4a2ba6-9109-4c7b-af4d-cae530c3fa78/recruitment-video-poster.jpg?trim=0%2C0%2C0%2C0&width=1000&quality=80',
     'og:image:alt': 'People chatting around a couch in a plant-filled sunny room',
     'twitter:card': 'summary_large_image',
     'twitter:site': '@OpenAI',
     'viewport': 

In [27]:
len(data['items'])

10

In [28]:
data['items'][0]

{'kind': 'customsearch#result',
 'title': 'OpenAI',
 'htmlTitle': '<b>OpenAI</b>',
 'link': 'https://openai.com/',
 'displayLink': 'openai.com',
 'snippet': 'Products · Customblogcover. Introducing improvements to the fine-tuning API and expanding our custom models program · Noauth Cover. Start using ChatGPT\xa0...',
 'htmlSnippet': 'Products &middot; Customblogcover. Introducing improvements to the fine-tuning API and expanding our custom models program &middot; Noauth Cover. Start using ChatGPT&nbsp;...',
 'cacheId': 'ZJjhy9NmZvMJ',
 'formattedUrl': 'https://openai.com/',
 'htmlFormattedUrl': 'https://<b>openai</b>.com/',
 'pagemap': {'metatags': [{'og:image': 'https://images.openai.com/blob/fb4a2ba6-9109-4c7b-af4d-cae530c3fa78/recruitment-video-poster.jpg?trim=0%2C0%2C0%2C0&width=1000&quality=80',
    'og:image:alt': 'People chatting around a couch in a plant-filled sunny room',
    'twitter:card': 'summary_large_image',
    'twitter:site': '@OpenAI',
    'viewport': 'width=device-w

In [29]:
def google_search(query, num_results=10, site_url=None):
    
    api_key = os.getenv("GOOGLE_SEARCH_API_KEY")
    cse_id = os.getenv("CSE_ID")
    
    url = "https://www.googleapis.com/customsearch/v1"

    # API 请求参数
    if site_url == None:
        params = {
        'q': query,          
        'key': api_key,      
        'cx': cse_id,        
        'num': num_results   
        }
    else:
        params = {
        'q': query,         
        'key': api_key,      
        'cx': cse_id,        
        'num': num_results,  
        'siteSearch': site_url
        }

    # 发送请求
    response = requests.get(url, params=params)
    response.raise_for_status()

    # 解析响应
    search_results = response.json().get('items', [])

    # 提取所需信息
    results = [{
        'title': item['title'],
        'link': item['link'],
        'snippet': item['snippet']
    } for item in search_results]

    return results

In [30]:
search_term = "OpenAI"

In [31]:
results = google_search(query=search_term, num_results=5)

In [32]:
results

[{'title': 'OpenAI',
  'link': 'https://openai.com/',
  'snippet': 'Products · Customblogcover. Introducing improvements to the fine-tuning API and expanding our custom models program · Noauth Cover. Start using ChatGPT\xa0...'},
 {'title': 'ChatGPT',
  'link': 'https://chat.openai.com/',
  'snippet': 'ChatGPT is a free-to-use AI system. Use it for engaging conversations, gain insights, automate tasks, and witness the future of AI, all in one place.'},
 {'title': 'Introducing ChatGPT',
  'link': 'https://openai.com/blog/chatgpt',
  'snippet': 'Nov 30, 2022 ... During the research preview, usage of ChatGPT is free. Try it now at chat.openai.com. Samples.'},
 {'title': 'OpenAI API: Overview',
  'link': 'https://platform.openai.com/',
  'snippet': "Explore resources, tutorials, API docs, and dynamic examples to get the most out of OpenAI's developer platform."},
 {'title': 'Sora',
  'link': 'https://openai.com/sora',
  'snippet': "Feb 15, 2024 ... We're sharing our research progress early

In [33]:
results = google_search(query='GLM4大模型有什么新特性？', num_results=10, site_url='https://www.zhihu.com/')

In [34]:
results

[{'title': '智谱发布GLM-4 基座大模型，性能效果如何？ - 知乎',
  'link': 'https://www.zhihu.com/question/639753877?write',
  'snippet': 'Jan 16, 2024 ... ... 大小时，路坚终于出现了。「我这边终于有新进展了，你在学校吗？我要当面告诉你！」室友激动坏了，突然高喊了一声「我赢了」，把我吓一跳。这个小镇\xa0...'},
 {'title': '智谱发布GLM-4 基座大模型，性能效果如何？ - 新智元的回答- 知乎',
  'link': 'https://www.zhihu.com/question/639753877/answer/3364585933',
  'snippet': 'Jan 15, 2024 ... 它能够支持更长的上下文，具备更强的多模态 功能，支持更快的推理，更多并发，推理成本大大降低。 同时，GLM-4也增强了智能体能力。 基础能力. 从众多评测\xa0...'},
 {'title': '智谱发布GLM-4 基座大模型，性能效果如何？ - 杨夕的回答- 知乎',
  'link': 'https://www.zhihu.com/question/639753877/answer/3364710951',
  'snippet': 'Jan 15, 2024 ... 众所周知，在我国众多大型模型企业中，我一直对智谱AI抱有极高的好感。 这不仅是因为他们浓厚的学术氛围…'},
 {'title': '智谱AI 宣布基座大模型GLM-4 发布，该大模型有何功能？ - 知乎',
  'link': 'https://www.zhihu.com/question/640050703',
  'snippet': 'Jan 17, 2024 ... 1月16日下午消息，智谱AI首届技术开放日上，智谱AI宣布发布新一代基座大模型GLM-4。据介绍，GLM-4的整体…'},
 {'title': '智谱发布GLM-4 基座大模型，性能效果如何？ - 大林的回答- 知乎',
  'link': 'https://www.zhihu.com/question/639753877/answer/3364780672',
  'snippet': 

## 4. 知乎网站数据爬取

### 4.1 数据格式定义

In [35]:
import json

file_path = 'result.json'

# 创建包含JSON对象的列表
json_data = [
    {
        "link": "https://example.com/article/1",
        "title": "Sample Article 1",
        "content": "This is the content of the first article...",
        "tokens": "Number of tokens..."
    }
]

# 指定保存文件的路径
file_path = "result.json"

# 将JSON数据写入本地文件
with open(file_path, "w", encoding="utf-8") as json_file:
    json.dump(json_data, json_file, ensure_ascii=False, indent=4)

print(f"JSON数据已保存到文件：{file_path}")

JSON数据已保存到文件：result.json


设置知乎排除的网站

In [36]:
results = google_search(query='GLM4大模型有什么新特性？', num_results=5, site_url='https://www.zhihu.com/')

In [37]:
results

[{'title': '智谱发布GLM-4 基座大模型，性能效果如何？ - 知乎',
  'link': 'https://www.zhihu.com/question/639753877?write',
  'snippet': 'Jan 16, 2024 ... ... 大小时，路坚终于出现了。「我这边终于有新进展了，你在学校吗？我要当面告诉你！」室友激动坏了，突然高喊了一声「我赢了」，把我吓一跳。这个小镇\xa0...'},
 {'title': '智谱发布GLM-4 基座大模型，性能效果如何？ - 新智元的回答- 知乎',
  'link': 'https://www.zhihu.com/question/639753877/answer/3364585933',
  'snippet': 'Jan 15, 2024 ... 它能够支持更长的上下文，具备更强的多模态 功能，支持更快的推理，更多并发，推理成本大大降低。 同时，GLM-4也增强了智能体能力。 基础能力. 从众多评测\xa0...'},
 {'title': '智谱发布GLM-4 基座大模型，性能效果如何？ - 杨夕的回答- 知乎',
  'link': 'https://www.zhihu.com/question/639753877/answer/3364710951',
  'snippet': 'Jan 15, 2024 ... 众所周知，在我国众多大型模型企业中，我一直对智谱AI抱有极高的好感。 这不仅是因为他们浓厚的学术氛围…'},
 {'title': '智谱AI 宣布基座大模型GLM-4 发布，该大模型有何功能？ - 知乎',
  'link': 'https://www.zhihu.com/question/640050703',
  'snippet': 'Jan 17, 2024 ... 1月16日下午消息，智谱AI首届技术开放日上，智谱AI宣布发布新一代基座大模型GLM-4。据介绍，GLM-4的整体…'},
 {'title': '智谱发布GLM-4 基座大模型，性能效果如何？ - 大林的回答- 知乎',
  'link': 'https://www.zhihu.com/question/639753877/answer/3364780672',
  'snippet': 

### 4.2 网络爬虫代码编写

In [38]:
## 请求头
headers = {
    'authority': 'www.zhihu.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8',
    'cache-control': 'max-age=0',
    'cookie': "Your cookie",                # 需要手动获取cookie
    'upgrade-insecure-requests': '1',
    'user-agent': 'Your user-agent',        # 手动编写或者选择之后给出的user-agent选项选择其一填写
}

1. **authority**: 通常表示请求的目标主机名。在当前项目中需要视情况填写`www.zhihu.com`或者`zhuanlan.zhihu.com`；

2. **accept**: 告诉服务器客户端能够处理的内容类型。这里列出了多种内容类型，包括HTML、XML和各种图片格式，该参数按照给定内容填写即可；

3. **accept-language**: 告诉服务器客户端的首选语言。这里，首选语言被设置为简体中文（zh-CN），其次是其他中文版本（zh），然后是英文（en）。同样该参数的内容按照给定内容填写即可；

4. **cache-control**: 控制缓存的行为。`max-age=0`通常表示客户端不希望得到缓存的响应，而希望从原始服务器获取一个新的响应。该参数的内容按照给定内容填写即可；

5. **cookie**: 包含服务器之前发送给客户端的cookie。这些cookie可能用于身份验证、会话跟踪或其他目的。这里需要重点注意，建议大家自行获取对应个人浏览器产生的cookie，以避免cookie滥用导致被识别为机器人从而导致封IP；

6. **upgrade-insecure-requests**: 这个头部告诉服务器，如果可能的话，客户端希望使用更安全的协议（如HTTPS）进行通信。建议取值为1即可；

7. **user-agent**: 描述发出请求的客户端的类型。这里，它模拟了一个Chrome浏览器的用户代理字符串。网站有时会根据这个头部提供不同的内容或布局，或者检测是否是爬虫。该参数也需要根据自己实际情况进行编写，接下来会提供一些可选的客户端模拟参数，大家可以选择其中一个作为自己的user-agent。

In [138]:
cookie = "q_c1=3c10baf5bd084b3cbfe7eece648ba243|1704976541000|1704976541000; _zap=086350b3-1588-49c4-9d6d-de88f3faae03; d_c0=AGCYfYvO_RePTvjfB1kZwPLeke_N5AM6nwo=|1704949678; _xsrf=qR1FJHlZ9dvYhhoj4SUj43SAIBUwPOqm; __snaker__id=wNWnamiJKBI0kzkI; q_c1=d44e397edb6740859a7d2a0d4155bfab|1706509753000|1706509753000; Hm_lvt_98beee57fd2ef70ccdd5ca52b9740c49=1706509695,1706589529,1706765465,1708650963; z_c0=2|1:0|10:1713167971|4:z_c0|80:MS4xOGRHQVNnQUFBQUFtQUFBQVlBSlZUYjZqOTJaLXVDSjdRSmJKMHgyVEhxTE13UGN1TUJBdHZnPT0=|15b2c2ece393ac4ea374d9b36cde5af7304f8ee7632e060fe6835bfadb5e4132; KLBRSID=9d75f80756f65c61b0a50d80b4ca9b13|1713170212|1713167958"
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"

In [40]:
## 请求头
headers = {
    'authority': 'www.zhihu.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8',
    'cache-control': 'max-age=0',
    'cookie': cookie,                # 需要手动获取cookie
    'upgrade-insecure-requests': '1',
    'user-agent': user_agent,        # 手动编写或者选择之后给出的user-agent选项选择其一填写
}

In [41]:
## question类型的网站
url = 'https://www.zhihu.com/question/639753877'

In [42]:
res = requests.get(url, headers=headers).text

In [43]:
res

'<!doctype html>\n<html lang="zh" data-hairline="true" class="itcauecng" data-theme="light"><head><meta charSet="utf-8"/><title data-rh="true">智谱发布 GLM-4 基座大模型，性能效果如何？ - 知乎</title><meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1"/><meta name="renderer" content="webkit"/><meta name="force-rendering" content="webkit"/><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"/><meta name="google-site-verification" content="FTeR0c8arOPKh8c5DYh_9uu98_zJbaWw53J-Sch9MTg"/><meta data-rh="true" name="description" property="og:description" content="量子位：国产GPTs来了，基于智谱第4代大模型！模型性能均达GPT-4九成以上"/><meta data-rh="true" name="keywords" content="人工智能,大模型,LLM（大型语言模型）,AI先行者,智谱"/><link data-rh="true" rel="apple-touch-icon" href="https://static.zhihu.com/heifetz/assets/apple-touch-icon-152.81060cab.png"/><link data-rh="true" rel="apple-touch-icon" href="https://static.zhihu.com/heifetz/assets/apple-touch-icon-152.81060cab.png" sizes="152x152"/><link data-rh="true" rel="appl

In [44]:
res_xpath = etree.HTML(res)

In [45]:
res_xpath

<Element html at 0x272630f2b80>

In [46]:
title = res_xpath.xpath('//div/div[1]/div/h1/text()')[0]

In [47]:
title

'智谱发布 GLM-4 基座大模型，性能效果如何？'

In [48]:
## 获取回答的正文
#//*[@id="root"]/div/main/div/div/div[1]/div[2]/div/div[1]/div[1]/div[6]/div/div/div/div/span/p[1]/text()
text = ''
text_d = res_xpath.xpath('//div/div/div/div[2]/div/div[2]/div/div/div[2]/span[1]/div/div/span/p/text()')
for t in text_d:
    txt = str(t).replace('\n', ' ')
    text +=txt

print(text)

我简单试了一下，确实跟GPT4几乎一模一样的技术栈。我上传了这么一张图片能读出来，也比较准，看来OCR做的不错。换了另一张图，是sine正弦函数的可视化，最简单的可视化。首先结果不对，这是正弦函数，y的结果一眼不对。第二点，不如GPT4的code interpreter可以执行代码。但是我觉得这个大模型算是国内的很不错的大模型了，起码gpt4有的它都有，虽然是打折版的。并且使用也免费，已经挺够意思了。我只是有一点儿担忧，就是毕竟大模型的最大的护城河还是在模型的智能程度上，OpenAI以下的大模型其实大家都半斤八两，没有谁很突出。希望国内大模型也能尽快赶上。


完成了question类的网站的爬取，接下来保存爬虫的结果

In [49]:
text

'我简单试了一下，确实跟GPT4几乎一模一样的技术栈。我上传了这么一张图片能读出来，也比较准，看来OCR做的不错。换了另一张图，是sine正弦函数的可视化，最简单的可视化。首先结果不对，这是正弦函数，y的结果一眼不对。第二点，不如GPT4的code interpreter可以执行代码。但是我觉得这个大模型算是国内的很不错的大模型了，起码gpt4有的它都有，虽然是打折版的。并且使用也免费，已经挺够意思了。我只是有一点儿担忧，就是毕竟大模型的最大的护城河还是在模型的智能程度上，OpenAI以下的大模型其实大家都半斤八两，没有谁很突出。希望国内大模型也能尽快赶上。'

In [50]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [51]:
len(encoding.encode(text))

285

In [52]:
json_data = [
    {
        "link": url,
        "title": title,
        "content": text,
        "tokens": len(encoding.encode(text))
    }
]

In [53]:
title="智谱发布GLM-4基座大模型，性能效果如何"

In [56]:
with open('./auto_search/介绍一下GLM4大模型的特性/%s.json' % title, 'w') as f:
    json.dump(json_data, f)

In [57]:
with open('./auto_search/介绍一下GLM4大模型的特性/%s.json' % title, 'r') as f:
    jd = json.load(f)

In [58]:
jd

[{'link': 'https://www.zhihu.com/question/639753877',
  'title': '智谱发布 GLM-4 基座大模型，性能效果如何？',
  'content': '我简单试了一下，确实跟GPT4几乎一模一样的技术栈。我上传了这么一张图片能读出来，也比较准，看来OCR做的不错。换了另一张图，是sine正弦函数的可视化，最简单的可视化。首先结果不对，这是正弦函数，y的结果一眼不对。第二点，不如GPT4的code interpreter可以执行代码。但是我觉得这个大模型算是国内的很不错的大模型了，起码gpt4有的它都有，虽然是打折版的。并且使用也免费，已经挺够意思了。我只是有一点儿担忧，就是毕竟大模型的最大的护城河还是在模型的智能程度上，OpenAI以下的大模型其实大家都半斤八两，没有谁很突出。希望国内大模型也能尽快赶上。',
  'tokens': 285}]

In [59]:
## question/answer网站
url = 'https://www.zhihu.com/question/639787253/answer/3365580226'
res = requests.get(url, headers=headers).text
res_xpath = etree.HTML(res)

In [60]:
# question/answer网站标题
res_xpath.xpath('//div/div[1]/div/h1/text()')[0]

'如何看待智谱AI发布GLM4？国产大模型与GPT-4更加接近了吗？'

In [61]:
# question/answer网站置顶回答内容
dic = []
text_d = res_xpath.xpath('//*[@id="root"]/div/main/div/div/div[3]/div[1]/div/div[2]/div/div/div/div[2]/span[1]/div/div/span/p/text()')
for t in text_d:
    txt = str(t).replace('\n', ' ')
    dic.append(txt)
    print(txt)

没想到，2024年开年，国内大模型首个传来好消息的竟是智谱AI。
就在昨日的首届技术开放日上，
在大规模多任务语言理解评测中，GLM-4得分远超GPT-3.5，平均达到GPT-4的95%的水平，个别项目上已几乎持平；
在GSM8K数学评测数据集上，GLM-4的评测结果与GPT-4有4.6%的差距；在MATH数据集上，GLM-4得分比GPT-3.5多15%，比GPT-4少9%。
GLM-4可
同时，其技术团队解决了上下文全局信息因失焦而导致的精度下降问题，在“大海捞针”测试中，GLM-4模型能做到几乎100%的精准召回。
推出全新的
在对齐、保真、安全、组合布局等各个评测维度上，CogView3都做到DALL·E 3 90%以上的水平，平均可以达到95%的相对性能。
针对令许多开发者及普通用户犯难的“AI咒语”提示词问题，
，可以说是“国内的GPTs”。
它实现了根据用户意图自动理解、规划复杂指令，自由调用文生图、代码解释器、网页浏览、Function Call等多项工具来完成复杂任务。
，不用再为提示词而担心。
在GLM4发布的那段时间，
在最新发布第三代基础大语言模型ChatGLM3系列。官方表示该模型的性能较前一代大幅提升，是10B以下最强基础大模型。
具体来看，
但是前面8个模型最小的也是140亿参数规模的Qwen-14B，如果按照GSM8K排序，ChatGLM3-6B-Base甚至排到第三，超过了GPT-3.5的57.1分。
可能对于非AI从业者来说，2023年是大模型的元年，
就拿智谱来说，早在大模型露出苗头时，它就已经做了技术上的预研或者成立新公司专注大模型，而不是在看到ChatGPT之后，才下定决心要做大模型。
一直以来，国产大模型例如文心一言就对标ChatGPT，甚至是对标GPT-4，门槛极高。但国产大模型在追赶ChatGPT之路上，其实是没有终点的长跑。
人工智能的市场很大，每个公司都可以在其中找到自己的位置。
在这场AI竞赛中，
前段时间爆火的AI扩图，给我们带来了观察图片画面的新角度。这个软件就是
此外，它还提供了AI抠图等一系列智能修图功能。
首页进入之后，就可以看到【
】功能。操作玩法也非常简单，
这就是AI扩图出来的效果，不仅是增加像素的数量，更重要的是增加图片分辨率。
百度旗下的一款AI视频创作工具，它的目的为打造人人可用的全流程AI创

In [62]:
# 专栏类网站
url = 'https://zhuanlan.zhihu.com/p/469793124'
res = requests.get(url, headers=headers).text
res_xpath = etree.HTML(res)

In [63]:
# 专栏类网站标题
res_xpath.xpath('//div[1]/div/main/div/article/header/h1/text()')[0]

'一文了解指令集和微架构'

In [64]:
# 专栏类网站标题内容
dic = []
text_d = res_xpath.xpath('//div/main/div/article/div[1]/div/div/div/p/text()')
for t in text_d:
    txt = str(t).replace('\n', ' ')
    dic.append(txt)
    print(txt)

建设一栋大楼的基本元素有：水泥、砖头、钢材等原材料，但不同的构造过程则会呈现不同的建筑形式，其中建造图纸起到了很大的指导作用。
同样， CPU从表象上看虽然形式很多，但基本电路都由晶体管构成，例如常见的MOS管。
通过晶体管等基本电子元器件的组合可构成基本的逻辑电路：如与门、非门、与非门等。
这些基本逻辑电路通过不同的逻辑组合可分别完成不同的功能，就好比“把大象放进冰箱的段子”，首先打开冰箱门à然后把大象放进去à最后关上冰箱门。通过这些逻辑组合使动作具有了意义，而这些实现特定功能的逻辑组合集合就是指令集，如基本的加减运算。
指令集是一个标准，其会随着需求变化不断添加新的指令或优化。同样，指令集发生变更后，工程师在设计CPU时也需要在硬件电路上增加对应的电路模块来支持变更的指令，配套的编译器也会随之升级。我们在玩51、STM32等单片机时需要相应的编译器来实现相应程序开发就是这个道理。而在任何一款遵循同一指令集架构实现的处理器上，开发的应用无须做任何修改便可以运行。
无论处于上层的应用程序多么酷炫吊炸天，其在处理器上执行就必须被翻译成“机器语言”，然后通过0或1的排列组合去操作硬件实现功能。翻译官这个角色就是编译器的活，它将软件与硬件世界连接了起来。编译器在这个过程中，要经过编译、汇编、链接等几个步骤,最后生成"可执行文件"，可执行文件中保存的是二进制机器码，这串机器码可以直接被CPU读取和执行。
设计一款CPU的过程就好比建房子，我们建房子肯定先要以一定的建设标准和规范为前提，依据一定的工程标准不仅使建设过程有序进行更重要的是最终能被验收和被大家所接受。这也类似于我们软件开发流程，必须先有需求再有开发，否则开发的软件都没有应用的场景。因此凭空设计一款CPU，首先会不会被厂商用另说，很多相应的配套也难以支持更何谈让软件开发人员进行开发和推广了。所以指令集作为一种标准规范，用于规范芯片设计工程师及编译器开发工程师。
因为芯片与集成开发环境-IDE都遵循相同的指令集标准，所以高级语言编写的程序经指定编译器编译后能直接运行在对应的CPU上，反之则不能运行。
所以，CPU在设计之前，就需要先设计一套指令集或者说使用现成的指令集(如ARM、X86指令集)并在硬件电路上实现这些指令。CPU设计好后，还需要配套的编译器，编译器也需要参考这个指令集标准，将我们编写的C程序、C++等程

### 4.3 爬取知乎网站的代码逻辑封装

In [65]:
url = 'https://www.zhihu.com/question/639787253/answer/3365580226'

In [66]:
if 'answer' in url:
    print('a')

a


In [67]:
def get_search_text(q, url):

    cookie = "q_c1=3c10baf5bd084b3cbfe7eece648ba243|1704976541000|1704976541000; _zap=086350b3-1588-49c4-9d6d-de88f3faae03; d_c0=AGCYfYvO_RePTvjfB1kZwPLeke_N5AM6nwo=|1704949678; _xsrf=qR1FJHlZ9dvYhhoj4SUj43SAIBUwPOqm; __snaker__id=wNWnamiJKBI0kzkI; q_c1=d44e397edb6740859a7d2a0d4155bfab|1706509753000|1706509753000; Hm_lvt_98beee57fd2ef70ccdd5ca52b9740c49=1706509695,1706589529,1706765465,1708650963; z_c0=2|1:0|10:1713167971|4:z_c0|80:MS4xOGRHQVNnQUFBQUFtQUFBQVlBSlZUYjZqOTJaLXVDSjdRSmJKMHgyVEhxTE13UGN1TUJBdHZnPT0=|15b2c2ece393ac4ea374d9b36cde5af7304f8ee7632e060fe6835bfadb5e4132; KLBRSID=9d75f80756f65c61b0a50d80b4ca9b13|1713170212|1713167958"
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"  

    code_ = False
    headers = {
        'authority': 'www.zhihu.com',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8',
        'cache-control': 'max-age=0',
        'cookie': cookie,
        'upgrade-insecure-requests': '1',
        'user-agent':user_agent,
    }

    # 普通问答地址
    if 'zhihu.com/question' in url:
        res = requests.get(url, headers=headers).text
        res_xpath = etree.HTML(res)
        title = res_xpath.xpath('//div/div[1]/div/h1/text()')[0]
        text_d = res_xpath.xpath('//*[@id="root"]/div/main/div/div/div[3]/div[1]/div/div[2]/div/div/div/div[2]/span[1]/div/div/span/p/text()')
    
    # 专栏地址
    elif 'zhuanlan' in url:
        headers['authority'] = 'zhaunlan.zhihu.com'
        res = requests.get(url, headers=headers).text
        res_xpath = etree.HTML(res)
        title = res_xpath.xpath('//div[1]/div/main/div/article/header/h1/text()')[0]
        text_d = res_xpath.xpath('//div/main/div/article/div[1]/div/div/div/p/text()')
        code_ = res_xpath.xpath('//div/main/div/article/div[1]/div/div/div//pre/code/text()')  
            
    # 特定回答的问答网址
    elif 'answer' in url:
        res = requests.get(url, headers=headers).text
        res_xpath = etree.HTML(res)
        title = res_xpath.xpath('//div/div[1]/div/h1/text()')[0]
        text_d = res_xpath.xpath('//div[1]/div/div[3]/div/div/div/div[2]/span[1]/div/div/span/p/text()')

    # 创建问题答案正文
    text = ''
    for t in text_d:
        txt = str(t).replace('\n', ' ')
        text += txt
        
    # 如果有code，则将code追加到正文的追后面
    if code_:
        for c in code_:
            co = str(c).replace('\n', ' ')    
            text += co
    
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")     
    json_data = [
        {
            "link": url,
            "title": title,
            "content": text,
            "tokens": len(encoding.encode(text))
        }
    ]

    with open('./auto_search/%s/%s.json' % (q, title), 'w') as f:
        json.dump(json_data, f)
    
    return title

In [68]:
url = 'https://www.zhihu.com/question/639787253/answer/3365580226'

In [69]:
title = get_search_text(q='介绍一下GLM4大模型的特性', url=url)
title

'如何看待智谱AI发布GLM4？国产大模型与GPT-4更加接近了吗？'

In [70]:
with open('./auto_search/介绍一下GLM4大模型的特性/%s.json' % '如何看待智谱AI发布GLM4？国产大模型与GPT-4更加接近了吗？', 'r') as f:
    jd = json.load(f)

In [71]:
jd

[{'link': 'https://www.zhihu.com/question/639787253/answer/3365580226',
  'title': '如何看待智谱AI发布GLM4？国产大模型与GPT-4更加接近了吗？',
  'content': '没想到，2024年开年，国内大模型首个传来好消息的竟是智谱AI。就在昨日的首届技术开放日上，在大规模多任务语言理解评测中，GLM-4得分远超GPT-3.5，平均达到GPT-4的95%的水平，个别项目上已几乎持平；在GSM8K数学评测数据集上，GLM-4的评测结果与GPT-4有4.6%的差距；在MATH数据集上，GLM-4得分比GPT-3.5多15%，比GPT-4少9%。GLM-4可同时，其技术团队解决了上下文全局信息因失焦而导致的精度下降问题，在“大海捞针”测试中，GLM-4模型能做到几乎100%的精准召回。推出全新的在对齐、保真、安全、组合布局等各个评测维度上，CogView3都做到DALL·E 3 90%以上的水平，平均可以达到95%的相对性能。针对令许多开发者及普通用户犯难的“AI咒语”提示词问题，，可以说是“国内的GPTs”。它实现了根据用户意图自动理解、规划复杂指令，自由调用文生图、代码解释器、网页浏览、Function Call等多项工具来完成复杂任务。，不用再为提示词而担心。在GLM4发布的那段时间，在最新发布第三代基础大语言模型ChatGLM3系列。官方表示该模型的性能较前一代大幅提升，是10B以下最强基础大模型。具体来看，但是前面8个模型最小的也是140亿参数规模的Qwen-14B，如果按照GSM8K排序，ChatGLM3-6B-Base甚至排到第三，超过了GPT-3.5的57.1分。可能对于非AI从业者来说，2023年是大模型的元年，就拿智谱来说，早在大模型露出苗头时，它就已经做了技术上的预研或者成立新公司专注大模型，而不是在看到ChatGPT之后，才下定决心要做大模型。一直以来，国产大模型例如文心一言就对标ChatGPT，甚至是对标GPT-4，门槛极高。但国产大模型在追赶ChatGPT之路上，其实是没有终点的长跑。人工智能的市场很大，每个公司都可以在其中找到自己的位置。在这场AI竞赛中，前段时间爆火的AI扩图，给我们带来了观察图片画面的新角度。这个软件就是此外，它还提供了AI抠图等

### 4.5 自动搜索流程封装

In [75]:
def get_answer(q):
    """
    当你无法回答某个问题时，调用该函数，能够获得答案
    :param q: 必选参数，询问的问题，字符串类型对象
    :return：某问题的答案，以字符串形式呈现
    """
    # 默认搜索返回10个答案（我这儿为了节省资源改成3）
    results = google_search(query=q, num_results=3, site_url='https://zhihu.com/')
    
    # 创建对应问题的子文件夹
    folder_path = './auto_search/%s' % q
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # 单独提取links放在一个list中
    num_tokens = 0
    content = ''
    for item in results:
        url = item['link']
        title = get_search_text(q, url)
        with open('./auto_search/%s/%s.json' % (q, title), 'r') as f:
            jd = json.load(f)
        num_tokens += jd[0]['tokens']
        if num_tokens <= 12000:
            content += jd[0]['content']
        else:
            break
    return(content)

In [76]:
q = '介绍一下GPT-3.5的微调方法'

In [77]:
qa = get_answer(q)
qa

'一夜之间，大模型界又炸出个big news！斯坦福发布（羊驼，网友口中的“草泥马”）：还有一个更绝的“骚操作”。研究所涉及到的数据集，是斯坦福团队花了不到500美元用OpenAI的API来生成的。所以整个过程下来，就等同于GPT-3.5自己教出了个旗鼓相当的对手AI。（薅羊毛高手……）然后团队还说，用大多数云计算平台去微调训练好的模型，成本也不到100美元：而且团队还把数据集（秒省500刀）、代码统统都给开源了，这下子人人都能去微调个效果炸裂的对话AI：项目在GitHub发布才半天时间，便已经狂揽1800+星，火爆程度可见一斑。Django联合开发者甚至对斯坦福的新研究用“惊天大事”来形容：不仅如此，斯坦福团队还搞了个demo，在线可玩的那种。话不多说，我们现在就来看看这个“草泥马”的效果。在斯坦福官方的演示中，他们先小试牛刀地提了一个问题：草泥马Aplaca给出的答案较为干练：而后又简单的介绍了二者群居生活的不同。同样的问题若是交给ChatGPT（GPT3.5-turbo），则答案就不会像草泥马Aplaca那般简洁：对此，团队给出的解释是：而后团队演示了让草泥马Alpaca：草泥马Alpaca对于这个任务也是信手拈来，直接给出了一个像模像样的邮件模板：难度再次进阶，团队这次提出了让草泥马Alpaca的需求：草泥马Alpaca给出的答案从内容上来看，非常符合大多数论文的摘要形式：试图回答什么问题、用了什么方法、结果如何，以及未来展望。当然，也有迫不及待的网友亲自下场试验，发现草泥马Alpaca写代码也是不在话下。不过即便草泥马Alpaca能够hold住大部分问题，但这并不意味着它没有缺陷。例如团队便演示了一个例子，在回答“坦桑尼亚的首都是哪里”的问题时，草泥马Alpaca给出的答案是“达累斯萨拉姆”。但实际上早在1975年便被“多多马”取代了。除此之外，若是亲自体验过草泥马Alpaca就会发现，它……巨慢：对此，有网友认为可能是使用的人太多的原因。Meta开源的LLaMA大模型，刚发布几周就被大家安排明白了，单卡就能运行。所以理论上，基于LLaMA微调的Alpaca同样可以轻松在本地部署。没有显卡也没关系，苹果笔记本甚至树莓派、手机都可以玩。在苹果笔记本部署LLaMA的方法来自GitHub项目llama.cpp，使用纯C/C++做推理，还专门对ARM芯片做了优化。

In [78]:
response = client.chat.completions.create(
  model="glm-4",
  messages=[
    {"role": "system", "content": qa},
    {"role": "user", "content": 'GPT-3.5的微调方法'}
  ]
)
response.choices[0].message.content

'根据提供的信息，GPT-3.5的微调方法主要包括以下几点：\n\n1. **GPT-3.5 Turbo的微调**：OpenAI最近宣布，所有开发者都可以对GPT-3.5 Turbo进行微调，以便更好地满足特定需求。通过微调，开发者可以增强模型在特定任务中的表现，提高其可控性和输出的一致格式。\n\n2. **微调益处**：微调后的GPT-3.5 Turbo在特定任务中的表现与GPT-4相当，甚至有所超越。微调可以增加模型的个性化、效率和灵活性，让模型更好地适应特定需求。\n\n3. **微调方法**：GPT-3.5的微调可以通过OpenAI提供的Web界面进行，无需编程知识，用户可以直接在网页上进行微调操作。同时，OpenAI也发布了GPT-3.5的微调API，为开发者提供了微调GPT-3.5的接口。\n\n4. **微调效果**：初步结果表明，通过微调，模型在遵循指令、输出格式一致性等方面取得了显著提升。微调后，模型可以更精确地完成特定任务，提高效率。\n\n5. **应用前景**：GPT-3.5的微调为开发者提供了根据需求定制化模型的能力，预计将催生大量新应用，标志着OpenAI在AI商业应用方面迈出了新的一步。\n\n总体来说，GPT-3.5的微调方法旨在让开发者能够根据特定需求对模型进行定制，从而提高其适用性，为用户创造更独特和差异化的体验。'

整合Funcation Calling流程

In [84]:
def get_answer(q):
    """
    智能助手函数，当你无法回答某个问题时，调用该函数，能够获得答案
    :param q: 必选参数，询问的问题，字符串类型对象
    :return：某问题的答案，以字符串形式呈现
    """
    print("使用谷歌搜索")
    # 默认搜索返回3个答案
    results = google_search(query=q, num_results=3, site_url='https://zhihu.com/')
    
    # 创建对应问题的子文件夹
    folder_path = './auto_search/%s' % q
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # 单独提取links放在一个list中
    num_tokens = 0
    content = ''
    for item in results:
        url = item['link']
        title = get_search_text(q, url)
        with open('./auto_search/%s/%s.json' % (q, title), 'r') as f:
            jd = json.load(f)
        num_tokens += jd[0]['tokens']
        if num_tokens <= 12000:
            content += jd[0]['content']
        else:
            break
    return json.dumps({"Question": q, "Answer": content})

In [80]:
q = '介绍一下GPT-3.5的微调方法'

In [81]:
get_answer(q)

'{"Question": "\\u4ecb\\u7ecd\\u4e00\\u4e0bGPT-3.5\\u7684\\u5fae\\u8c03\\u65b9\\u6cd5", "Answer": "\\u4e00\\u591c\\u4e4b\\u95f4\\uff0c\\u5927\\u6a21\\u578b\\u754c\\u53c8\\u70b8\\u51fa\\u4e2abig news\\uff01\\u65af\\u5766\\u798f\\u53d1\\u5e03\\uff08\\u7f8a\\u9a7c\\uff0c\\u7f51\\u53cb\\u53e3\\u4e2d\\u7684\\u201c\\u8349\\u6ce5\\u9a6c\\u201d\\uff09\\uff1a\\u8fd8\\u6709\\u4e00\\u4e2a\\u66f4\\u7edd\\u7684\\u201c\\u9a9a\\u64cd\\u4f5c\\u201d\\u3002\\u7814\\u7a76\\u6240\\u6d89\\u53ca\\u5230\\u7684\\u6570\\u636e\\u96c6\\uff0c\\u662f\\u65af\\u5766\\u798f\\u56e2\\u961f\\u82b1\\u4e86\\u4e0d\\u5230500\\u7f8e\\u5143\\u7528OpenAI\\u7684API\\u6765\\u751f\\u6210\\u7684\\u3002\\u6240\\u4ee5\\u6574\\u4e2a\\u8fc7\\u7a0b\\u4e0b\\u6765\\uff0c\\u5c31\\u7b49\\u540c\\u4e8eGPT-3.5\\u81ea\\u5df1\\u6559\\u51fa\\u4e86\\u4e2a\\u65d7\\u9f13\\u76f8\\u5f53\\u7684\\u5bf9\\u624bAI\\u3002\\uff08\\u8585\\u7f8a\\u6bdb\\u9ad8\\u624b\\u2026\\u2026\\uff09\\u7136\\u540e\\u56e2\\u961f\\u8fd8\\u8bf4\\uff0c\\u7528\\u5927\\u591a\\u6

In [87]:
run_conversation(messages=[{"role": "user", "content": '介绍一下你自己？'}], 
                 functions_list=[get_answer], 
                 model="glm-4")

'我是一名虚拟助手,我被训练来回答各种问题,提供帮助和建议。我并不是真正的人,我只是一个计算机程序,但是我可以尽可能地模仿人类的语言和思维方式来与您交流。'

In [95]:
run_conversation(messages=[
    {"role": "system", "content": "根据用户输入的问题进行回答，如果知道问题的答案，请回答问题答案，如果不知道问题答案，调用智能助手函数回答’"},
    {"role": "user", "content": '介绍一下什么是机器学习？'}
    ], 
    functions_list=[get_answer], 
    model="glm-4")

'机器学习是一门人工智能（AI）的分支，它使计算机系统能够从数据中学习并做出决策或预测，而无需每一步都进行明确编程。通过训练算法解析数据，机器学习模型能够识别数据中的模式，从而对新的数据点做出预测或决策。机器学习在许多领域都有应用，包括图像和语音识别、自然语言处理、医疗诊断、金融预测等。'

In [90]:
run_conversation(messages=[
    {"role": "system", "content": "根据用户输入的问题进行回答，如果知道问题的答案，请回答问题答案，如果不知道问题答案，调用智能助手函数回答’"},
    {"role": "user", "content": '今天的北京的天气是多少度'}
    ], 
    functions_list=[get_answer], 
    model="glm-4")

使用谷歌搜索
content='北京今天的天气是多云，气温大约在-11℃到-3℃之间。' role='assistant' tool_calls=None


'北京今天的天气是多云，气温大约在-11℃到-3℃之间。'

## 5. 流程优化

### 5.1 文件名优化

文件名里有特殊字符的，在创建文件的时候会创建失败

In [97]:
def windows_create_name(s, max_length=255):
    """
    将字符串转化为符合Windows文件/文件夹命名规范的名称。
    
    参数:
    - s (str): 输入的字符串。
    - max_length (int): 输出字符串的最大长度，默认为255。
    
    返回:
    - str: 一个可以安全用作Windows文件/文件夹名称的字符串。
    """

    # Windows文件/文件夹名称中不允许的字符列表
    forbidden_chars = ['<', '>', ':', '"', '/', '\\', '|', '?', '*']

    # 使用下划线替换不允许的字符
    for char in forbidden_chars:
        s = s.replace(char, '_')

    # 删除尾部的空格或点
    s = s.rstrip(' .')

    # 检查是否存在以下不允许被用于文档名称的关键词，如果有的话则替换为下划线
    reserved_names = ["CON", "PRN", "AUX", "NUL", "COM1", "COM2", "COM3", "COM4", "COM5", "COM6", "COM7", "COM8", "COM9", 
                      "LPT1", "LPT2", "LPT3", "LPT4", "LPT5", "LPT6", "LPT7", "LPT8", "LPT9"]
    if s.upper() in reserved_names:
        s += '_'

    # 如果字符串过长，进行截断
    if len(s) > max_length:
        s = s[:max_length]

    return s

In [98]:
# 测试
input_str = "这是一个包含不允许的字符，如 <, >, :, \, /, |, ?, * 等的很长的字符串..."
print(windows_create_name(input_str))

这是一个包含不允许的字符，如 _, _, _, _, _, _, _, _ 等的很长的字符串


In [139]:
def get_search_text(q, url):

    cookie = "q_c1=3c10baf5bd084b3cbfe7eece648ba243|1704976541000|1704976541000; _zap=086350b3-1588-49c4-9d6d-de88f3faae03; d_c0=AGCYfYvO_RePTvjfB1kZwPLeke_N5AM6nwo=|1704949678; _xsrf=qR1FJHlZ9dvYhhoj4SUj43SAIBUwPOqm; __snaker__id=wNWnamiJKBI0kzkI; q_c1=d44e397edb6740859a7d2a0d4155bfab|1706509753000|1706509753000; Hm_lvt_98beee57fd2ef70ccdd5ca52b9740c49=1706509695,1706589529,1706765465,1708650963; z_c0=2|1:0|10:1713167971|4:z_c0|80:MS4xOGRHQVNnQUFBQUFtQUFBQVlBSlZUYjZqOTJaLXVDSjdRSmJKMHgyVEhxTE13UGN1TUJBdHZnPT0=|15b2c2ece393ac4ea374d9b36cde5af7304f8ee7632e060fe6835bfadb5e4132; KLBRSID=9d75f80756f65c61b0a50d80b4ca9b13|1713170212|1713167958"
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    title = None
    
    code_ = False
    headers = {
        'authority': 'www.zhihu.com',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8',
        'cache-control': 'max-age=0',
        'cookie': cookie,
        'upgrade-insecure-requests': '1',
        'user-agent':user_agent,
    }

    # 普通问答地址
    if 'zhihu.com/question' in url:
        res = requests.get(url, headers=headers).text
        res_xpath = etree.HTML(res)
        title = res_xpath.xpath('//div/div[1]/div/h1/text()')[0]
        text_d = res_xpath.xpath('//div/div/div/div[2]/div/div[2]/div/div/div[2]/span[1]/div/div/span/p/text()')
    
    # 专栏地址
    elif 'zhuanlan' in url:
        headers['authority'] = 'zhaunlan.zhihu.com'
        res = requests.get(url, headers=headers).text
        res_xpath = etree.HTML(res)
        title = res_xpath.xpath('//div[1]/div/main/div/article/header/h1/text()')[0]
        text_d = res_xpath.xpath('//div/main/div/article/div[1]/div/div/div/p/text()')
        code_ = res_xpath.xpath('//div/main/div/article/div[1]/div/div/div//pre/code/text()')  
            
    # 特定回答的问答网址
    elif 'answer' in url:
        res = requests.get(url, headers=headers).text
        res_xpath = etree.HTML(res)
        title = res_xpath.xpath('//div/div[1]/div/h1/text()')[0]
        text_d = res_xpath.xpath('//div[1]/div/div[3]/div/div/div/div[2]/span[1]/div/div/span/p/text()')

    if title == None:
        return None
    
    else:
        title = windows_create_name(title)

        # 创建问题答案正文
        text = ''
        for t in text_d:
            txt = str(t).replace('\n', ' ')
            text += txt

        # 如果有code，则将code追加到正文的追后面
        if code_:
            for c in code_:
                co = str(c).replace('\n', ' ')    
                text += co

        encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")     
        json_data = [
            {
                "link": url,
                "title": title,
                "content": text,
                "tokens": len(encoding.encode(text))
            }
        ]

        with open('./auto_search/%s/%s.json' % (q, title), 'w') as f:
            json.dump(json_data, f)

        return title

### 5.2 构建判别模型

为了让大模型调用流程更稳定，创建一个判别模型

In [100]:
import random
import string

def generate_random_key(length=30):
    return ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(length))

In [103]:
def identify_model(q):
    
    # 创建密钥
    sk = generate_random_key()
    
    # 调用模型进行判别
    response = client.chat.completions.create(
            model="glm-4",
            messages=[
                {"role": "system", "content": "你是一个用户问题判断器，专门用于判别你是否知道用户当前问题的答案。\
                如果不知道，请回答“%s”，若知道，请正常回答" % sk},
                {"role": "user", "content": "请问，GPT-3.5微调总共分为几步？"},
                {"role": "assistant", "content": "%s" % sk},
                {"role": "user", "content": q}
            ]
        )
    res = response.choices[0].message.content
    
    if sk in res or '对不起' in res or '抱歉' in res or '超出知识库' in res:
        return(True)
    else:
        return(res)

In [104]:
q = '请问什么是机器学习？'
print(identify_model(q))

机器学习是人工智能（AI）的一个分支，它赋予计算机从数据中学习的能力，使其能够通过经验改进处理任务的能力。机器学习算法利用统计学方法，让计算机系统在不需要进行显式编程的情况下，通过识别数据中的模式和特征来自动改进其性能。它广泛应用于各种领域，包括语音识别、图像处理、医学诊断、推荐系统等。机器学习主要包括监督学习、非监督学习、半监督学习和强化学习等几种方法。


In [105]:
q = '介绍一下北京今天的天气'
print(identify_model(q))  

True


In [109]:
q = '介绍一下DB-GPT项目？'
print(identify_model(q))

DB-GPT是一个开源项目，旨在通过结合大型语言模型（LLMs）和数据库技术，实现数据处理的自动化和优化。这个项目的核心是利用本地部署的大模型来与数据和环境进行交互，以此保护数据隐私和安全。DB-GPT支持在独立私有环境中部署，可以根据业务需求进行模块化部署，确保大模型的能力在私域内是绝对私有、安全和可控的。

该项目主要具有以下特点：

1. **数据安全和隐私保护**：通过本地化部署，避免了数据泄露的风险，确保用户数据的安全和隐私。
   
2. **全流程自动化**：整合了数据摄取、结构化和访问等多个流程，通过增强型大型语言模型来优化这些流程。

3. **多模型框架**：基于RAG框架设计，采用了多源RAG、自适应上下文学习等技术，提供了面向服务的多模型框架。

4. **知识库构建与处理**：原生支持知识库的构建和处理，通过langchain提供私域知识库问答能力。

5. **插件模式**：支持Auto-GPT等插件，提高了与数据交互的效率。

6. **文本转SQL细化调整**：包含文本转SQL任务的语言模型，能够更好地理解和处理自然语言查询。

7. **多源知识整合**：能够整合多源知识库，支持双语查询和生成数据分析。

DB-GPT的架构包括多个关键模块，例如用于知识构建、知识检索、自适应交互式编程和响应生成的模块，这些模块协同工作，以提高性能和效率。

项目的代码在GitHub上开源，地址是：[https://github.com/eosphoros-ai/DB-GPT](https://github.com/eosphoros-ai/DB-GPT)。该项目鼓励社区参与和贡献，共同推动数据库与大型语言模型结合的技术发展。


### 5.3 搜索词优化

用户提出的问题并不一定是最适合在知乎上进行搜索的关键词，例如用户提问“我想要知道“GPT-3.5微调总共需要几步”，此时应该将其转化为更加准确凝练的关键词并输入到知乎上进行搜索，方可提高搜索结果准确性。这里我们也完全可以让大模型自主将用户提的问题转化为适合在知乎上进行搜索的搜索关键词。当然，为达此目的，我们需要首先测试大模型是否知道知乎，以及大模型是否知道如何设计搜索关键词：

In [110]:
response = client.chat.completions.create(
        model="glm-4",
        messages=[
            {"role": "user", "content": "你知道什么是知乎么？"}
        ]
    )
response.choices[0].message.content

'当然知道。知乎是中国领先的知识分享和社交平台，以问答的形式构建了一个信息分享、经验交流、意见互动的在线社区。它由北京智者天下科技有限公司运营，成立于2011年1月26日，其名称寓意在文言文中“知道吗”。\n\n知乎的目标是构建一个高质量的问答社区，让用户可以找到解决问题的答案，同时也能分享自己的知识与见解。随着时间的推移，知乎逐渐发展成为一个综合性内容平台，不仅提供问答服务，还涵盖了图文、音频、视频等多种内容形式。\n\n用户可以在知乎上提出问题，其他用户则可以回答问题或者对已有的回答进行评论。知乎的社区氛围以认真、专业和友善著称，吸引了大量专业人士和领域专家参与讨论。\n\n根据提供的信息，知乎在2019年完成了F轮融资，并推出了包括“盐选会员”在内的多项服务。到了2024年，知乎进一步拓展了其业务范围，不仅在社区建设上持续发力，还推出了结合AI技术的功能，如“发现·AI搜索”，以及“海盐计划6.0：航海家”和“灯塔计划”等项目，旨在通过技术赋能，提升用户体验，并推动平台持续进化。\n\n截至2020年12月，知乎拥有超过4400万条问题和2.4亿条回答，月活跃付费用户数超过250万，显示了其强大的用户基础和商业潜力。知乎不仅是一个问答社区，更是一个内容创作和知识分享的重要平台。'

In [111]:
response = client.chat.completions.create(
        model="glm-4",
        messages=[
            {"role": "user", "content": "我如果想在知乎上搜索一些问题的答案，在设计搜索关键词方面，有什么技巧么？"}
        ]
    )
response.choices[0].message.content

'在设计搜索关键词以便在知乎上找到满意的问题答案时，以下是一些有用的技巧：\n\n1. **明确搜索意图**：\n   - 在开始搜索前，明确你想要解决的问题或者你感兴趣的话题。\n   - 确定你的搜索目标是寻找具体信息、教程、观点对比还是案例分析。\n\n2. **选择准确的关键词**：\n   - 选择与问题主题密切相关的关键词。\n   - 使用专业术语或行话，如果适用，以获取更精准的结果。\n   - 考虑使用同义词或近义词，以扩大搜索范围。\n\n3. **组合关键词**：\n   - 使用组合关键词来提高搜索的针对性。例如，如果你的问题是关于“数据分析工具”，可以组合成“数据分析工具比较”、“数据分析软件推荐”等。\n   - 尝试不同的组合方式，如“最佳”、“如何”、“教程”、“区别”等，以找到不同角度的答案。\n\n4. **利用长尾关键词**：\n   - 长尾关键词通常更具体，竞争也较小，可以帮助你找到更精确的信息。例如，“Python数据分析入门教程”就比单独的“Python”更具体。\n\n5. **避免过度堆砌关键词**：\n   - 过度堆砌关键词可能导致搜索结果不准确，降低搜索效率。\n\n6. **利用知乎的搜索提示**：\n   - 知乎的搜索栏通常会有相关搜索提示，可以参考这些提示来调整和优化你的关键词。\n\n7. **注意关键词的顺序**：\n   - 有时关键词的顺序也会影响搜索结果，尝试不同的顺序来获得更准确的搜索结果。\n\n8. **检查拼写和语法**：\n   - 确保关键词的拼写和语法正确，以免影响搜索效果。\n\n9. **使用高级搜索选项**：\n   - 如果知乎支持高级搜索，利用这些功能，如时间筛选、用户筛选等，来进一步优化搜索结果。\n\n通过上述技巧，你可以设计出更有效的搜索关键词，提高在知乎上找到高质量答案的几率。'

In [112]:
response = client.chat.completions.create(
        model="glm-4",
        messages=[
            {"role": "user", "content": "例如，我想要知道“GPT-3.5微调总共需要几步”，\
            现在围绕这个问题在知乎上进行搜索，应该怎样设计这个问题的搜索关键词呢？"}
        ]
    )
response.choices[0].message.content

'若要在知乎上搜索关于“GPT-3.5微调总共需要几步”的信息，可以设计以下关键词进行搜索：\n\n1. GPT-3.5 微调流程\n2. GPT-3.5定制步骤\n3. GPT-3.5模型训练几步走\n4. GPT-3.5 Turbo 微调指南\n5. GPT-3.5自定义模型步骤\n6. GPT-3.5训练数据生成与微调流程\n7. GPT-3.5 RAG混合方法步骤\n\n以下是一些可能的搜索短语：\n\n- “GPT-3.5微调需要哪些步骤？”\n- “如何对GPT-3.5进行定制训练？”\n- “GPT-3.5从数据生成到微调的流程介绍”\n- “OpenAI GPT-3.5微调流程详解”\n- “GPT-3.5 Turbo训练与微调步骤”\n\n组合这些关键词和短语，可以更准确地找到关于GPT-3.5微调步骤的相关讨论和答案。在搜索时，也可以根据需要添加一些限定词，比如“2023”，“最新指南”等，以获取最新的信息。'

In [113]:
response = client.chat.completions.create(
        model="glm-4",
        messages=[
             {"role": "system", "content": "你专门负责将用户的问题转化为知乎网站搜索关键词，只返回一个你认为最合适的搜索关键词即可"},
             {"role": "user", "content": "请问，GPT-3.5微调总共分为几步？"},
             {"role": "assistant", "content": "GPT-3.5微调流程"},
             {"role": "user", "content": "请问ChatGPT企业版都有哪些功能？"}
        ]
    )
response.choices[0].message.content

'ChatGPT Enterprise 功能列表'

In [114]:
def convert_keyword(q):
    """
    将用户输入的问题转化为适合在知乎上进行搜索的关键词
    """
    response = client.chat.completions.create(
            model="glm-4",
            messages=[
                 {"role": "system", "content": "你专门负责将用户的问题转化为知乎网站搜索关键词，只返回一个你认为最合适的搜索关键词即可"},
                 {"role": "user", "content": "请问，GPT-3.5微调总共分为几步？"},
                 {"role": "assistant", "content": "GPT-3.5微调流程"},
                 {"role": "user", "content": q}
            ]
        )
    q = response.choices[0].message.content
    return q

In [116]:
def get_answer(q):
    """
    智能助手函数，当你无法回答某个问题时，调用该函数，能够获得答案
    :param q: 必选参数，询问的问题，字符串类型对象
    :return：某问题的答案，以字符串形式呈现
    """
    # 调用转化函数，将用户的问题转化为更适合在知乎上进行搜索的关键词
    q = convert_keyword(q)
    
    # 默认搜索返回10个答案
    print('正在接入谷歌搜索，查找和问题相关的答案...')
    results = google_search(query=q, num_results=2, site_url='https://zhihu.com/')
    
    # 创建对应问题的子文件夹
    folder_path = './auto_search/%s' % q
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # 单独提取links放在一个list中
    print('正在读取搜索的到的相关答案...')
    num_tokens = 0
    content = ''
    for item in results:
        url = item['link']
        title = get_search_text(q, url)
        with open('./auto_search/%s/%s.json' % (q, title), 'r') as f:
            jd = json.load(f)
        num_tokens += jd[0]['tokens']
        if num_tokens <= 12000:
            content += jd[0]['content']
        else:
            break
    print('正在进行最后的整理...')
    return(content)

### 5.4 外部函数流程优化

而既然是依据“密钥”来判断是否调用搜索功能，那么该外部函数的调用就不能再使用此前定义的auto function calling功能了，而是需要我们手动来调用这个外部函数（即接收到密钥时一定要调用外部函数）

In [118]:
functions_list=[get_answer]

In [119]:
tools = auto_functions(functions_list)
tools

[{'type': 'function',
  'function': {'name': 'get_answer',
   'description': 'When unable to answer a question, the function is called to obtain an answer.',
   'parameters': {'type': 'object',
    'properties': {'q': {'type': 'string',
      'description': 'The question to be asked.'}},
    'required': ['q'],
    'additionalProperties': False}}}]

In [121]:
response = client.chat.completions.create(
        model="glm-4",
        messages=[
            {"role": "user", "content": "今天的北京天气如何？"},
            
        ],
        tools=tools,
        tool_choice={"type": "function", "function": {"name": "get_answer"}},  
    )
response.choices[0].message

CompletionMessage(content=None, role='assistant', tool_calls=[CompletionMessageToolCall(id='call_8578778912776223368', function=Function(arguments='{"q":"北京今天天气"}', name='get_answer'), type='function')])

In [131]:
def run_conversation(messages, functions_list=None, model="glm-4", function_call="auto"):
    """
    能够自动执行外部函数调用的对话模型
    :param messages: 必要参数，字典类型，输入到Chat模型的messages参数对象
    :param functions_list: 可选参数，默认为None，可以设置为包含全部外部函数的列表对象
    :param model: Chat模型，可选参数，默认模型为glm-4
    :return：Chat模型输出结果
    """
    # 如果没有外部函数库，则执行普通的对话任务
    if functions_list == None:
        response = client.chat.completions.create(
                        model=model,
                        messages=messages,
                        )
        response_message = response.choices[0].message
        final_response = response_message.content
        
    # 若存在外部函数库，则需要灵活选取外部函数并进行回答
    else:
        # 创建functions对象
        tools = auto_functions(functions_list)

        # 创建外部函数库字典
        available_functions = {func.__name__: func for func in functions_list}

        # 第一次调用大模型
        response = client.chat.completions.create(
                        model=model,
                        messages=messages,
                        tools=tools,
                        tool_choice=function_call, )
        response_message = response.choices[0].message


        tool_calls = response_message.tool_calls

        if tool_calls:

            #messages.append(response_message)
            messages.append(response.choices[0].message.model_dump())
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)
                ## 真正执行外部函数的就是这儿的代码
                function_response = function_to_call(**function_args)
                messages.append(
                    {
                        "role": "tool",
                        "content": function_response,
                        "tool_call_id": tool_call.id,
                    }
                ) 
            ## 第二次调用模型
            second_response = client.chat.completions.create(
                model=model,
                messages=messages,
                tools=tools
            ) 
            # 获取最终结果
            print(second_response.choices[0].message)
            final_response = second_response.choices[0].message.content
        else:
            final_response = response_message.content
                
    return final_response

优化后的智能机器人逻辑编写

In [132]:
def auto_search_answer(q):
    # 调用判别模型
    res = identify_model(q)
    if res == True:
        messages = [{"role": "user", "content": q}]
        res =run_conversation(messages=messages, 
                              functions_list=[get_answer], 
                              model="glm-4", 
                              function_call={"type": "function", "function": {"name": "get_answer"}})
    return(res)

In [140]:
auto_search_answer("北京2024年5月1日的天气如何？")

正在接入谷歌搜索，查找和问题相关的答案...
正在读取搜索的到的相关答案...
正在进行最后的整理...
content='很抱歉，目前无法预测北京2024年5月1日的天气。' role='assistant' tool_calls=None


'很抱歉，目前无法预测北京2024年5月1日的天气。'

In [141]:
auto_search_answer("什么是大模型？")

'大模型指的是参数规模极大、计算能力极强、能够处理非常复杂任务的人工智能模型。通常这些模型的参数量达到百亿、千亿甚至更多，例如GPT-3、天工3.0等。大模型具有卓越的理解能力、生成能力和泛化能力，能够应用于多种场景，如图像识别、自然语言处理、音乐生成等，为人工智能在各个领域的应用提供了强大的支持。随着算力的提升和算法的发展，大模型在推动人工智能技术进步方面发挥着越来越重要的作用。'

In [142]:
auto_search_answer("abab 6.5模型是什么时候推出来的？")

'abab 6.5模型是在2024年4月17日推出的。'

### 5.5 全流程代码总结 

In [ ]:
import os
import openai
from openai import OpenAI
import glob
import shutil

import numpy as np
import pandas as pd

import json
import io
import inspect
import requests
import re
import random
import string

from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
import base64


from bs4 import BeautifulSoup
import dateutil.parser as parser
import tiktoken
from lxml import etree

## 初始化客户端
api_key = os.getenv("ZHIPUAI_API_KEY")


from zhipuai import ZhipuAI
client = ZhipuAI(api_key=api_key)

In [ ]:
def google_search(query, num_results=10, site_url=None):
    
    api_key = os.getenv("GOOGLE_SEARCH_API_KEY")
    cse_id = os.getenv("CSE_ID")
    
    url = "https://www.googleapis.com/customsearch/v1"

    # API 请求参数
    if site_url == None:
        params = {
        'q': query,          
        'key': api_key,      
        'cx': cse_id,        
        'num': num_results   
        }
    else:
        params = {
        'q': query,         
        'key': api_key,      
        'cx': cse_id,        
        'num': num_results,  
        'siteSearch': site_url
        }

    # 发送请求
    response = requests.get(url, params=params)
    response.raise_for_status()

    # 解析响应
    search_results = response.json().get('items', [])

    # 提取所需信息
    results = [{
        'title': item['title'],
        'link': item['link'],
        'snippet': item['snippet']
    } for item in search_results]

    return results

In [ ]:
def windows_create_name(s, max_length=255):
    """
    将字符串转化为符合Windows文件/文件夹命名规范的名称。
    
    参数:
    - s (str): 输入的字符串。
    - max_length (int): 输出字符串的最大长度，默认为255。
    
    返回:
    - str: 一个可以安全用作Windows文件/文件夹名称的字符串。
    """

    # Windows文件/文件夹名称中不允许的字符列表
    forbidden_chars = ['<', '>', ':', '"', '/', '\\', '|', '?', '*']

    # 使用下划线替换不允许的字符
    for char in forbidden_chars:
        s = s.replace(char, '_')

    # 删除尾部的空格或点
    s = s.rstrip(' .')

    # 检查是否存在以下不允许被用于文档名称的关键词，如果有的话则替换为下划线
    reserved_names = ["CON", "PRN", "AUX", "NUL", "COM1", "COM2", "COM3", "COM4", "COM5", "COM6", "COM7", "COM8", "COM9", 
                      "LPT1", "LPT2", "LPT3", "LPT4", "LPT5", "LPT6", "LPT7", "LPT8", "LPT9"]
    if s.upper() in reserved_names:
        s += '_'

    # 如果字符串过长，进行截断
    if len(s) > max_length:
        s = s[:max_length]

    return s

In [ ]:
def get_search_text(q, url):

    cookie = "q_c1=3c10baf5bd084b3cbfe7eece648ba243|1704976541000|1704976541000; _zap=086350b3-1588-49c4-9d6d-de88f3faae03; d_c0=AGCYfYvO_RePTvjfB1kZwPLeke_N5AM6nwo=|1704949678; _xsrf=qR1FJHlZ9dvYhhoj4SUj43SAIBUwPOqm; __snaker__id=wNWnamiJKBI0kzkI; q_c1=d44e397edb6740859a7d2a0d4155bfab|1706509753000|1706509753000; Hm_lvt_98beee57fd2ef70ccdd5ca52b9740c49=1706509695,1706589529,1706765465,1708650963; z_c0=2|1:0|10:1713167971|4:z_c0|80:MS4xOGRHQVNnQUFBQUFtQUFBQVlBSlZUYjZqOTJaLXVDSjdRSmJKMHgyVEhxTE13UGN1TUJBdHZnPT0=|15b2c2ece393ac4ea374d9b36cde5af7304f8ee7632e060fe6835bfadb5e4132; KLBRSID=9d75f80756f65c61b0a50d80b4ca9b13|1713170212|1713167958"
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"  

    code_ = False
    headers = {
        'authority': 'www.zhihu.com',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8',
        'cache-control': 'max-age=0',
        'cookie': cookie,
        'upgrade-insecure-requests': '1',
        'user-agent':user_agent,
    }

    # 普通问答地址
    if 'zhihu.com/question' in url:
        res = requests.get(url, headers=headers).text
        res_xpath = etree.HTML(res)
        title = res_xpath.xpath('//div/div[1]/div/h1/text()')[0]
        text_d = res_xpath.xpath('//*[@id="root"]/div/main/div/div/div[3]/div[1]/div/div[2]/div/div/div/div[2]/span[1]/div/div/span/p/text()')
    
    # 专栏地址
    elif 'zhuanlan' in url:
        headers['authority'] = 'zhaunlan.zhihu.com'
        res = requests.get(url, headers=headers).text
        res_xpath = etree.HTML(res)
        title = res_xpath.xpath('//div[1]/div/main/div/article/header/h1/text()')[0]
        text_d = res_xpath.xpath('//div/main/div/article/div[1]/div/div/div/p/text()')
        code_ = res_xpath.xpath('//div/main/div/article/div[1]/div/div/div//pre/code/text()')  
            
    # 特定回答的问答网址
    elif 'answer' in url:
        res = requests.get(url, headers=headers).text
        res_xpath = etree.HTML(res)
        title = res_xpath.xpath('//div/div[1]/div/h1/text()')[0]
        text_d = res_xpath.xpath('//div[1]/div/div[3]/div/div/div/div[2]/span[1]/div/div/span/p/text()')

    # 创建问题答案正文
    text = ''
    for t in text_d:
        txt = str(t).replace('\n', ' ')
        text += txt
        
    # 如果有code，则将code追加到正文的追后面
    if code_:
        for c in code_:
            co = str(c).replace('\n', ' ')    
            text += co
    
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")     
    json_data = [
        {
            "link": url,
            "title": title,
            "content": text,
            "tokens": len(encoding.encode(text))
        }
    ]

    with open('./auto_search/%s/%s.json' % (q, title), 'w') as f:
        json.dump(json_data, f)
    
    return title

In [ ]:
import random
import string

def generate_random_key(length=30):
    return ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(length))

def identify_model(q):
    
    # 创建密钥
    sk = generate_random_key()
    
    # 调用模型进行判别
    response = client.chat.completions.create(
            model="glm-4",
            messages=[
                {"role": "system", "content": "你是一个用户问题判断器，专门用于判别你是否知道用户当前问题的答案。\
                如果不知道，请回答“%s”，若知道，请正常回答" % sk},
                {"role": "user", "content": "请问，GPT-3.5微调总共分为几步？"},
                {"role": "assistant", "content": "%s" % sk},
                {"role": "user", "content": q}
            ]
        )
    res = response.choices[0].message.content
    
    if sk in res or '对不起' in res or '抱歉' in res or '超出知识库' in res:
        return(True)
    else:
        return(res)

In [ ]:
def convert_keyword(q):
    """
    将用户输入的问题转化为适合在知乎上进行搜索的关键词
    """
    response = client.chat.completions.create(
            model="glm-4",
            messages=[
                 {"role": "system", "content": "你专门负责将用户的问题转化为知乎网站搜索关键词，只返回一个你认为最合适的搜索关键词即可"},
                 {"role": "user", "content": "请问，GPT-3.5微调总共分为几步？"},
                 {"role": "assistant", "content": "GPT-3.5微调流程"},
                 {"role": "user", "content": q}
            ]
        )
    q = response.choices[0].message.content
    return q

In [ ]:
def get_answer(q):
    """
    智能助手函数，当你无法回答某个问题时，调用该函数，能够获得答案
    :param q: 必选参数，询问的问题，字符串类型对象
    :return：某问题的答案，以字符串形式呈现
    """
    # 调用转化函数，将用户的问题转化为更适合在知乎上进行搜索的关键词
    q = convert_keyword(q)
    
    # 默认搜索返回10个答案
    print('正在接入谷歌搜索，查找和问题相关的答案...')
    results = google_search(query=q, num_results=2, site_url='https://zhihu.com/')
    
    # 创建对应问题的子文件夹
    folder_path = './auto_search/%s' % q
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # 单独提取links放在一个list中
    print('正在读取搜索的到的相关答案...')
    num_tokens = 0
    content = ''
    for item in results:
        url = item['link']
        title = get_search_text(q, url)
        with open('./auto_search/%s/%s.json' % (q, title), 'r') as f:
            jd = json.load(f)
        num_tokens += jd[0]['tokens']
        if num_tokens <= 12000:
            content += jd[0]['content']
        else:
            break
    print('正在进行最后的整理...')
    return(content)

In [ ]:
def auto_functions(functions_list):
    """
    Chat模型的functions参数编写函数
    :param functions_list: 包含一个或者多个函数对象的列表；
    :return：满足Chat模型functions参数要求的functions对象
    """
    def functions_generate(functions_list):
        # 创建空列表，用于保存每个函数的描述字典
        functions = []
        # 对每个外部函数进行循环
        for function in functions_list:
            # 读取函数对象的函数说明
            function_description = inspect.getdoc(function)
            # 读取函数的函数名字符串
            function_name = function.__name__

            system_prompt = '以下是某的函数说明：%s,输出结果必须是一个JSON格式的字典，只输出这个字典即可，前后不需要任何前后修饰或说明的语句' % function_description
            user_prompt = '根据这个函数的函数说明，请帮我创建一个JSON格式的字典，这个字典有如下5点要求：\
                           1.字典总共有三个键值对；\
                           2.第一个键值对的Key是字符串name，value是该函数的名字：%s，也是字符串；\
                           3.第二个键值对的Key是字符串description，value是该函数的函数的功能说明，也是字符串；\
                           4.第三个键值对的Key是字符串parameters，value是一个JSON Schema对象，用于说明该函数的参数输入规范。\
                           5.输出结果必须是一个JSON格式的字典，只输出这个字典即可，前后不需要任何前后修饰或说明的语句' % function_name

            response = client.chat.completions.create(
                              model="glm-4",
                              messages=[
                                {"role": "system", "content": system_prompt},
                                {"role": "user", "content": user_prompt}
                              ]
                            )
            json_str=response.choices[0].message.content.replace("```json","").replace("```","")
            json_function_description=json.loads(json_str)
            json_str={"type": "function","function":json_function_description}
            functions.append(json_str)
        return functions
    ## 最大可以尝试4次
    max_attempts = 4
    attempts = 0

    while attempts < max_attempts:
        try:
            functions = functions_generate(functions_list)
            break  # 如果代码成功执行，跳出循环
        except Exception as e:
            attempts += 1  # 增加尝试次数
            print("发生错误：", e)
            if attempts == max_attempts:
                print("已达到最大尝试次数，程序终止。")
                raise  # 重新引发最后一个异常
            else:
                print("正在重新运行...")
    return functions

In [ ]:
def run_conversation(messages, functions_list=None, model="glm-4", function_call="auto"):
    """
    能够自动执行外部函数调用的对话模型
    :param messages: 必要参数，字典类型，输入到Chat模型的messages参数对象
    :param functions_list: 可选参数，默认为None，可以设置为包含全部外部函数的列表对象
    :param model: Chat模型，可选参数，默认模型为glm-4
    :return：Chat模型输出结果
    """
    # 如果没有外部函数库，则执行普通的对话任务
    if functions_list == None:
        response = client.chat.completions.create(
                        model=model,
                        messages=messages,
                        )
        response_message = response.choices[0].message
        final_response = response_message.content
        
    # 若存在外部函数库，则需要灵活选取外部函数并进行回答
    else:
        # 创建functions对象
        tools = auto_functions(functions_list)

        # 创建外部函数库字典
        available_functions = {func.__name__: func for func in functions_list}

        # 第一次调用大模型
        response = client.chat.completions.create(
                        model=model,
                        messages=messages,
                        tools=tools,
                        tool_choice=function_call, )
        response_message = response.choices[0].message


        tool_calls = response_message.tool_calls

        if tool_calls:

            #messages.append(response_message)
            messages.append(response.choices[0].message.model_dump())
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)
                ## 真正执行外部函数的就是这儿的代码
                function_response = function_to_call(**function_args)
                messages.append(
                    {
                        "role": "tool",
                        "content": function_response,
                        "tool_call_id": tool_call.id,
                    }
                ) 
            ## 第二次调用模型
            second_response = client.chat.completions.create(
                model=model,
                messages=messages,
                tools=tools
            ) 
            # 获取最终结果
            print(second_response.choices[0].message)
            final_response = second_response.choices[0].message.content
        else:
            final_response = response_message.content
                
    return final_response

In [ ]:
def auto_search_answer(q):
    # 调用判别模型
    res = identify_model(q)
    if res == True:
        messages = [{"role": "user", "content": q}]
        res =run_conversation(messages=messages, 
                              functions_list=[get_answer], 
                              model="glm-4", 
                              function_call={"type": "function", "function": {"name": "get_answer"}})
    return(res)

## 6. 支持github网站在线搜索

### 6.1 github基础API测试

In [143]:
def convert_keyword_github(q):
    """
    将用户输入的问题转化为适合在Github上进行搜索的关键词
    """
    response = client.chat.completions.create(
            model="glm-4",
            messages=[
                 {"role": "system", "content": "你专门负责将用户的问题转化为Github上的搜索关键词，只返回一个你认为最合适的搜索关键词即可"},
                 {"role": "user", "content": "请问DeepSpeed是什么？"},
                 {"role": "assistant", "content": "DeepSpeed"},
                 {"role": "user", "content": q}
            ]
        )
    q = response.choices[0].message.content
    return q

In [144]:
convert_keyword_github('请帮我介绍下chatglm3大模型')

'ChatGLM3 大模型'

In [145]:
google_search(query='chatglm3', num_results=2, site_url='https://github.com/')

[{'title': 'ChatGLM3/README_en.md at main · THUDM/ChatGLM3 · GitHub',
  'link': 'https://github.com/THUDM/ChatGLM3/blob/main/README_en.md',
  'snippet': 'More Complete Function Support: ChatGLM3-6B adopts a newly designed Prompt format, supporting multi-turn dialogues as usual. It also natively supports tool\xa0...'},
 {'title': 'THUDM/ChatGLM3: ChatGLM3 series: Open Bilingual Chat ... - GitHub',
  'link': 'https://github.com/THUDM/ChatGLM3',
  'snippet': 'ChatGLM3 是智谱AI和清华大学KEG 实验室联合发布的对话预训练模型。ChatGLM3-6B 是ChatGLM3 系列中的开源模型，在保留了前两代模型对话流畅、部署门槛低等众多优秀特性的\xa0...'}]

In [146]:
## 获取github key
github_token = os.getenv('GITHUB_TOKEN')

In [147]:
import requests
import base64

owner = "THUDM"
repo = "chatglm3"

headers = {
    "Authorization": github_token,
    "User-Agent": user_agent
}

response = requests.get(f"https://api.github.com/repos/{owner}/{repo}/readme", headers=headers)

readme_data = response.json()
encoded_content = readme_data.get('content', '')
decoded_content = base64.b64decode(encoded_content).decode('utf-8')

print(decoded_content)

# ChatGLM3

<p align="center">
🤗 <a href="https://huggingface.co/THUDM/chatglm3-6b" target="_blank">HF Repo</a> • 🤖 <a href="https://modelscope.cn/models/ZhipuAI/chatglm3-6b" target="_blank">ModelScope</a> • 🤖 <a href="https://www.wisemodel.cn/models/ZhipuAI/chatglm3-6b" target="_blank">WiseModel</a> • 🧰 <a href="https://openxlab.org.cn/models/hot/THUDM" target="_blank">OpenXLab</a> • 🐦 <a href="https://twitter.com/thukeg" target="_blank">Twitter</a> • 📃 <a href="https://arxiv.org/abs/2103.10360" target="_blank">[GLM@ACL 22]</a> <a href="https://github.com/THUDM/GLM" target="_blank">[GitHub]</a> • 📃 <a href="https://arxiv.org/abs/2210.02414" target="_blank">[GLM-130B@ICLR 23]</a> <a href="https://github.com/THUDM/GLM-130B" target="_blank">[GitHub]</a> <br>
</p>
<p align="center">
    👋 加入我们的 <a href="https://join.slack.com/t/chatglm/shared_invite/zt-25ti5uohv-A_hs~am_D3Q8XPZMpj7wwQ" target="_blank">Slack</a> 和 <a href="resources/WECHAT.md" target="_blank">微信</a>
</p>
<p align="center">

In [148]:
def get_github_readme(dic):
    
    github_token = os.getenv('GITHUB_TOKEN')
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    
    owner = dic['owner']
    repo = dic['repo']

    headers = {
        "Authorization": github_token,
        "User-Agent": user_agent
    }

    response = requests.get(f"https://api.github.com/repos/{owner}/{repo}/readme", headers=headers)

    readme_data = response.json()
    encoded_content = readme_data.get('content', '')
    decoded_content = base64.b64decode(encoded_content).decode('utf-8')
    
    return decoded_content

In [149]:
dic = {'owner': 'THUDM', 'repo': 'chatglm3'}

In [150]:
get_github_readme(dic)

'# ChatGLM3\n\n<p align="center">\n🤗 <a href="https://huggingface.co/THUDM/chatglm3-6b" target="_blank">HF Repo</a> • 🤖 <a href="https://modelscope.cn/models/ZhipuAI/chatglm3-6b" target="_blank">ModelScope</a> • 🤖 <a href="https://www.wisemodel.cn/models/ZhipuAI/chatglm3-6b" target="_blank">WiseModel</a> • 🧰 <a href="https://openxlab.org.cn/models/hot/THUDM" target="_blank">OpenXLab</a> • 🐦 <a href="https://twitter.com/thukeg" target="_blank">Twitter</a> • 📃 <a href="https://arxiv.org/abs/2103.10360" target="_blank">[GLM@ACL 22]</a> <a href="https://github.com/THUDM/GLM" target="_blank">[GitHub]</a> • 📃 <a href="https://arxiv.org/abs/2210.02414" target="_blank">[GLM-130B@ICLR 23]</a> <a href="https://github.com/THUDM/GLM-130B" target="_blank">[GitHub]</a> <br>\n</p>\n<p align="center">\n    👋 加入我们的 <a href="https://join.slack.com/t/chatglm/shared_invite/zt-25ti5uohv-A_hs~am_D3Q8XPZMpj7wwQ" target="_blank">Slack</a> 和 <a href="resources/WECHAT.md" target="_blank">微信</a>\n</p>\n<p align=

In [151]:
search_results=google_search(query='chatglm3', num_results=2, site_url='https://github.com/')
search_results

[{'title': 'ChatGLM3/README_en.md at main · THUDM/ChatGLM3 · GitHub',
  'link': 'https://github.com/THUDM/ChatGLM3/blob/main/README_en.md',
  'snippet': 'More Complete Function Support: ChatGLM3-6B adopts a newly designed Prompt format, supporting multi-turn dialogues as usual. It also natively supports tool\xa0...'},
 {'title': 'THUDM/ChatGLM3: ChatGLM3 series: Open Bilingual Chat ... - GitHub',
  'link': 'https://github.com/THUDM/ChatGLM3',
  'snippet': 'ChatGLM3 是智谱AI和清华大学KEG 实验室联合发布的对话预训练模型。ChatGLM3-6B 是ChatGLM3 系列中的开源模型，在保留了前两代模型对话流畅、部署门槛低等众多优秀特性的\xa0...'}]

我们发现，GitHub上的项目主页网址的基本结构是`https://api.github.com/repos/{owner}/{repo}/readme `。其中owner就是项目的拥有者，而repo则是对应某个项目在Github上的名称。具体，如果我们希望在谷歌搜索获得的连接中进行逐个的readme获取，则首先需要筛选搜索结果中的项目主页链接，并且从中解析出每个连接的owner和repo

In [152]:
def extract_github_repos(search_results):
    # 使用列表推导式筛选出项目主页链接
    repo_links = [result['link'] for result in search_results if '/issues/' not in result['link'] and '/blob/' not in result['link'] and 'github.com' in result['link'] and len(result['link'].split('/')) == 5]

    # 从筛选后的链接中提取owner和repo
    repos_info = [{'owner': link.split('/')[3], 'repo': link.split('/')[4]} for link in repo_links]

    return repos_info

repos_info = extract_github_repos(search_results)
print(repos_info)

[{'owner': 'THUDM', 'repo': 'ChatGLM3'}]


### 6.2 完整流程封装

In [ ]:
def extract_github_repos(search_results):
    # 使用列表推导式筛选出项目主页链接
    repo_links = [result['link'] for result in search_results if '/issues/' not in result['link'] and '/blob/' not in result['link'] and 'github.com' in result['link'] and len(result['link'].split('/')) == 5]

    # 从筛选后的链接中提取owner和repo
    repos_info = [{'owner': link.split('/')[3], 'repo': link.split('/')[4]} for link in repo_links]

    return repos_info

In [154]:
def get_search_text_github(q, dic):
    
    title = dic['owner'] + '_' + dic['repo']
    title = windows_create_name(title)

    # 创建问题答案正文
    text = get_github_readme(dic)

    # 写入本地json文件
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")     
    json_data = [
        {
            "title": title,
            "content": text,
            "tokens": len(encoding.encode(text))
        }
    ]

    with open('./auto_search/%s/%s.json' % (q, title), 'w') as f:
        json.dump(json_data, f)

    return title

In [181]:
def get_answer_github(q):
    """
    智能助手函数，当你无法回答某个问题时，调用该函数，能够获得答案
    :param q: 必选参数，询问的问题，字符串类型对象
    :return：某问题的答案，以字符串形式呈现
    """
    # 调用转化函数，将用户的问题转化为更适合在GitHub上搜索的关键词
    q = convert_keyword_github(q)
    
    # 默认搜索返回10个答案
    print('正在接入谷歌搜索，查找和问题相关的答案...')
    search_results = google_search(query=q, num_results=10, site_url='https://github.com/')
    results = extract_github_repos(search_results)
    
    # 创建对应问题的子文件夹
    folder_path = './auto_search/%s' % q
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    print('正在读取搜索的到的相关答案...')
    num_tokens = 0
    content = ''
    
    for dic in results:
        title = get_search_text_github(q, dic)
        with open('./auto_search/%s/%s.json' % (q, title), 'r') as f:
            jd = json.load(f)
        num_tokens += jd[0]['tokens']
        if num_tokens <= 12000:
            content += jd[0]['content']
        else:
            break
    print('正在进行最后的整理...')
    return(content)

In [156]:
def auto_search_answer_github(q):
    # 调用判别模型
    res = identify_model(q)
    if res == True:
        messages = [{"role": "user", "content": q}]
        res =run_conversation(messages=messages, 
                              functions_list=[get_answer_github], 
                              model="glm-4", 
                              function_call={"type": "function", "function": {"name": "get_answer_github"}})
    return(res)

In [160]:
functions_list=[get_answer_github]

In [183]:
tools=auto_functions(functions_list)
tools

[{'type': 'function',
  'function': {'name': 'get_answer_github',
   'description': '当你无法回答某个问题时，调用该函数，能够获得答案',
   'parameters': {'type': 'object',
    'required': ['q'],
    'properties': {'q': {'type': 'string', 'description': '询问的问题，字符串类型对象'}},
    'additionalProperties': False}}}]

In [187]:
def run_conversation(messages, functions_list=None, model="glm-4", function_call="auto"):
    """
    能够自动执行外部函数调用的对话模型
    :param messages: 必要参数，字典类型，输入到Chat模型的messages参数对象
    :param functions_list: 可选参数，默认为None，可以设置为包含全部外部函数的列表对象
    :param model: Chat模型，可选参数，默认模型为glm-4
    :return：Chat模型输出结果
    """
    # 如果没有外部函数库，则执行普通的对话任务
    if functions_list == None:
        response = client.chat.completions.create(
                        model=model,
                        messages=messages,
                        )
        response_message = response.choices[0].message
        final_response = response_message.content
        
    # 若存在外部函数库，则需要灵活选取外部函数并进行回答
    else:
        # 创建functions对象
        tools = auto_functions(functions_list)

        # 创建外部函数库字典
        available_functions = {func.__name__: func for func in functions_list}

        # 第一次调用大模型
        response = client.chat.completions.create(
                        model=model,
                        messages=messages,
                        tools=tools,
                        tool_choice=function_call, )
        print(response)
        
        response_message = response.choices[0].message


        tool_calls = response_message.tool_calls

        if tool_calls:

            #messages.append(response_message)
            messages.append(response.choices[0].message.model_dump())
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)
                ## 真正执行外部函数的就是这儿的代码
                function_response = function_to_call(**function_args)
                messages.append(
                    {
                        "role": "tool",
                        "content": function_response,
                        "tool_call_id": tool_call.id,
                    }
                ) 
            ## 第二次调用模型
            second_response = client.chat.completions.create(
                model=model,
                messages=messages,
                tools=tools
            ) 
            # 获取最终结果
            print(second_response.choices[0].message)
            final_response = second_response.choices[0].message.content
        else:
            final_response = response_message.content
                
    return final_response

In [185]:
messages = [{"role": "system", "content": "对于知道的问题直接回答，不知道的问题，通过调用智能助手函数解答"},
            {"role": "user", "content": "介绍一下open-interpreter项目"}]
res =run_conversation(messages=messages, 
                              functions_list=[get_answer_github], 
                              model="glm-4", 
                              function_call={"type": "function", "function": {"name": "get_answer_github"}})
print(res)
## 

[{'type': 'function', 'function': {'name': 'get_answer_github', 'description': 'When unable to answer a question, this function can be called to obtain an answer.', 'parameters': {'type': 'object', 'required': ['q'], 'properties': {'q': {'type': 'string', 'description': 'The question to be asked.'}}, 'additionalProperties': False}}}]
---------------------------
glm-4
---------------------------
[{'role': 'system', 'content': '对于知道的问题直接回答，不知道的问题，通过调用智能助手函数解答'}, {'role': 'user', 'content': '介绍一下open-interpreter项目'}]
---------------------------
auto
model='glm-4' created=1713412685 choices=[CompletionChoice(index=0, finish_reason='tool_calls', message=CompletionMessage(content=None, role='assistant', tool_calls=[CompletionMessageToolCall(id='call_8578780871281458411', function=Function(arguments='{"q":"open-interpreter项目是什么？"}', name='get_answer_github'), type='function')]))] request_id='8578780871281458411' id='8578780871281458411' usage=CompletionUsage(prompt_tokens=142, completion_toke

In [197]:
messages = [{"role": "system", "content": "对于知道的问题直接回答，不知道的问题，通过调用智能助手函数解答"},
            {"role": "user", "content": "介绍一下github上面的open-interpreter项目"}]
res =run_conversation(messages=messages, 
                              functions_list=[get_answer_github], 
                              model="glm-4", 
                              function_call={"type": "function", "function": {"name": "get_answer_github"}})
print(res)

model='glm-4' created=1713412953 choices=[CompletionChoice(index=0, finish_reason='tool_calls', message=CompletionMessage(content=None, role='assistant', tool_calls=[CompletionMessageToolCall(id='call_8578779325093105020', function=Function(arguments='{"q":"介绍一下github上面的open-interpreter项目"}', name='get_answer_github'), type='function')]))] request_id='8578779325093105020' id='8578779325093105020' usage=CompletionUsage(prompt_tokens=142, completion_tokens=22, total_tokens=164)
正在接入谷歌搜索，查找和问题相关的答案...
正在读取搜索的到的相关答案...
正在进行最后的整理...
content='Open Interpreter是一个开源项目，它允许大型语言模型（LLM）在本地运行代码，支持Python、JavaScript、Shell等多种语言。通过自然语言界面，用户可以与计算机进行交互，执行各种任务，如创建和编辑图片、控制Chrome浏览器进行搜索、绘制和分析大型数据集等。Open Interpreter在执行代码前会要求用户确认，以避免潜在的安全风险。与ChatGPT的代码解释器相比，Open Interpreter运行在本地环境中，具有更广泛的功能和更高的灵活性。' role='assistant' tool_calls=None
Open Interpreter是一个开源项目，它允许大型语言模型（LLM）在本地运行代码，支持Python、JavaScript、Shell等多种语言。通过自然语言界面，用户可以与计算机进行交互，执行各种任务，如创建和编辑图片、控制Chrome浏览器进行搜索、绘制和分析大型数据集等。Open Interpreter在执行代码前会要求用户确认，以避免潜在的安

In [205]:
## 测试：网上随意找的一个项目：https://github.com/google/grafika
auto_search_answer_github('介绍一下github上面的Grafika项目，这个项目是Google发布的')

model='glm-4' created=1713413393 choices=[CompletionChoice(index=0, finish_reason='tool_calls', message=CompletionMessage(content=None, role='assistant', tool_calls=[CompletionMessageToolCall(id='call_8578779256373615457', function=Function(arguments='{"q":"介绍一下github上面的Grafika项目"}', name='get_answer_github'), type='function')]))] request_id='8578779256373615457' id='8578779256373615457' usage=CompletionUsage(prompt_tokens=127, completion_tokens=22, total_tokens=149)
正在接入谷歌搜索，查找和问题相关的答案...
正在读取搜索的到的相关答案...
正在进行最后的整理...
content='Grafika是一个由Google发布的项目，它是一个用于Android图形和媒体实验的代码仓库。这个项目包含了多种图形功能示例，使用Java语言编写，适用于API 18（Android 4.3）及以上版本。Grafika具有以下特点：\n\n- **功能丰富**：Grafika展示了多种图形功能，如播放视频、连续捕获、双解码、硬件缩放器、实时摄像头预览、多Surface测试、OpenGL ES渲染等。\n- **开源协议**：遵循Apache 2许可证，属于Google版权所有。\n- **持续更新**：Grafika是一个持续更新的项目，通常会针对平台的问题进行修复或提供解决方案。\n- **非正式产品**：Grafika不是一个官方的Google产品，而是由Google员工在公司时间和设备上开发的。\n\n虽然Grafika具有多种功能，但也存在一些限制：\n\n- **稳定性不足**：Grafika的稳定性不足，可能存在一些边缘情况未处理或处理不当。\n- **文档不完善**：Grafika的文档比较缺乏，可能

'Grafika是一个由Google发布的项目，它是一个用于Android图形和媒体实验的代码仓库。这个项目包含了多种图形功能示例，使用Java语言编写，适用于API 18（Android 4.3）及以上版本。Grafika具有以下特点：\n\n- **功能丰富**：Grafika展示了多种图形功能，如播放视频、连续捕获、双解码、硬件缩放器、实时摄像头预览、多Surface测试、OpenGL ES渲染等。\n- **开源协议**：遵循Apache 2许可证，属于Google版权所有。\n- **持续更新**：Grafika是一个持续更新的项目，通常会针对平台的问题进行修复或提供解决方案。\n- **非正式产品**：Grafika不是一个官方的Google产品，而是由Google员工在公司时间和设备上开发的。\n\n虽然Grafika具有多种功能，但也存在一些限制：\n\n- **稳定性不足**：Grafika的稳定性不足，可能存在一些边缘情况未处理或处理不当。\n- **文档不完善**：Grafika的文档比较缺乏，可能需要一定的开发经验才能理解和使用。\n- **不支持NDK**：Grafika完全使用Java语言编写，不使用NDK。\n- **非AOSP项目**：Grafika不是Android开源项目的一部分，不接受外部贡献。\n\n总的来说，Grafika是一个用于实验Android图形和媒体功能的工具集，适合有一定经验的开发者学习和参考。'

In [198]:
auto_search_answer_github('介绍一下github上面的DB-GPT项目')

'DB-GPT是一个基于数据库的开源GPT实验项目，旨在通过结合大型语言模型和数据库技术，实现在本地环境下对数据的自然语言查询和处理。这个项目的主要目的是解决在使用大模型时面临的数据安全和隐私问题。\n\n根据提供的参考信息，DB-GPT项目具有以下特点：\n\n1. **本地部署**：支持在个人设备或本地服务器上部署，可以在没有互联网连接的情况下运行，确保数据不会离开执行环境。\n\n2. **数据安全和隐私保护**：通过代理和插件机制优化数据驱动引擎，使用去标识化技术来保护用户隐私。\n\n3. **架构设计**：DB-GPT的系统架构包括检索器组件、重排器、语言模型处理精炼信息等，确保身份安全，并生成响应。\n\n4. **多源知识库**：系统可以使用来自数据库、网页、PDF文件等多个来源的信息。\n\n5. **插件模式**：原生支持Auto-GPT插件，以及其他代理，如AIOps代理、SQL代理和商业分析代理，以便与不同类型的数据和服务接口。\n\n6. **核心能力**：支持知识库的构建与处理，使用FastChat构建大模型运行环境，以vicuna作为基础的大语言模型，并通过langchain提供私域知识库问答能力。\n\n项目的代码和文档在GitHub上开放，地址为：[https://github.com/eosphoros-ai/DB-GPT](https://github.com/eosphoros-ai/DB-GPT)。\n\nDB-GPT项目的目标是简化围绕数据库的大模型应用构建，同时保证数据处理的安全性和可控性。这对于需要在保护数据隐私和安全的条件下使用大模型的企业和组织来说，是一个非常吸引人的解决方案。'

In [ ]:
def chat_with_model(functions_list=None, 
                    prompt="你好", 
                    model="glm-4", 
                    system_message=[{"role": "system", "content": "你是小智助手。"}]):
    
    messages = system_message
    messages.append({"role": "user", "content": prompt})
    
    while True:           
        answer = run_conversation(messages=messages, 
                              functions_list=functions_list, 
                              model=model, 
                              function_call={"type": "function", "function": {"name": "get_answer_github"}})
        
        print(f"智能助手回答: {answer}")
        
        
        # 询问用户是否还有其他问题
        user_input = input("您还有其他问题吗？(输入退出以结束对话): ")
        if user_input == "退出":
            break

        # 记录用户回答
        messages.append({"role": "user", "content": user_input})

## 7. 完成HuggingFace网站搜索

将Hugging face接入GPT模型中。得益于Hugging face完善的API体系，相比前两个智能搜索问答项目，将Hugging face接入GPT模型的过程会简单很多，甚至都不用谷歌搜索API即可完成。总的来说，将Hugging face接入GPT模型的流程是，首先，借助此前定义的convert_keyword_github函数提取用户提问的模型关键词，然后借助Hugging face的Search API完成模型搜索，和Github类似，在Hugging face中，同一个模型会有多种类型的变种，这里我们可以搜索得到一些较为受欢迎的模型进行模型功能的查询。而具体的查询过程也同样是查询不同模型的readme文档，而Hugging face上的readme文档同样也是可以通过API快速获取。

### 7.1 HuggingFace API调用开发

In [206]:
def search_models(query):
    url = "https://huggingface.co/api/models"
    params = {"search": query}
    response = requests.get(url, params=params)
    return response.json()

search_results = search_models("chatglm")

In [207]:
len(search_results)

412

In [208]:
## 我们根据模型的受欢迎程度（likes指标）对其进行降序排序，挑选较为受欢迎的模型进行问答：
sorted_results = sorted(search_results, key=lambda x: x['likes'], reverse=True)

In [209]:
sorted_results[:10]

[{'_id': '640f4f1409c94e1d9bca3ffc',
  'id': 'THUDM/chatglm-6b',
  'likes': 2773,
  'private': False,
  'downloads': 26319,
  'tags': ['transformers',
   'pytorch',
   'chatglm',
   'glm',
   'thudm',
   'custom_code',
   'zh',
   'en',
   'arxiv:2103.10360',
   'arxiv:2210.02414',
   'endpoints_compatible',
   'has_space',
   'region:us'],
  'library_name': 'transformers',
  'createdAt': '2023-03-13T16:28:04.000Z',
  'modelId': 'THUDM/chatglm-6b'},
 {'_id': '64971933a9c42bc6a848d3f4',
  'id': 'THUDM/chatglm2-6b',
  'likes': 1995,
  'private': False,
  'downloads': 100996,
  'tags': ['transformers',
   'pytorch',
   'chatglm',
   'glm',
   'thudm',
   'custom_code',
   'zh',
   'en',
   'arxiv:2103.10360',
   'arxiv:2210.02414',
   'arxiv:1911.02150',
   'endpoints_compatible',
   'has_space',
   'region:us'],
  'library_name': 'transformers',
  'createdAt': '2023-06-24T16:26:27.000Z',
  'modelId': 'THUDM/chatglm2-6b'},
 {'_id': '6538e659a608e1c3a212cb75',
  'id': 'THUDM/chatglm3-6b',


In [210]:
sorted_results[2]['id']

'THUDM/chatglm3-6b'

### 7.2 获取readme文档

In [211]:
## 接下来即可根据模型ID，调用对应Hugging face API来查询各模型的说明文档。具体实现过程如下
def get_model_readme(model_id):
    url = f"https://huggingface.co/{model_id}/resolve/main/README.md"
    response = requests.get(url)
    return response.text

In [212]:
model_readme = get_model_readme(sorted_results[1]['id'])

In [213]:
model_readme

'---\nlanguage:\n- zh\n- en\ntags:\n- glm\n- chatglm\n- thudm\n---\n# ChatGLM2-6B\n<p align="center">\n  💻 <a href="https://github.com/THUDM/ChatGLM2-6B" target="_blank">Github Repo</a> • 🐦 <a href="https://twitter.com/thukeg" target="_blank">Twitter</a> • 📃 <a href="https://arxiv.org/abs/2103.10360" target="_blank">[GLM@ACL 22]</a> <a href="https://github.com/THUDM/GLM" target="_blank">[GitHub]</a> • 📃 <a href="https://arxiv.org/abs/2210.02414" target="_blank">[GLM-130B@ICLR 23]</a> <a href="https://github.com/THUDM/GLM-130B" target="_blank">[GitHub]</a> <br>\n</p>\n\n<p align="center">\n    👋 Join our <a href="https://join.slack.com/t/chatglm/shared_invite/zt-1y7pqoloy-9b1g6T6JjA8J0KxvUjbwJw" target="_blank">Slack</a> and <a href="https://github.com/THUDM/ChatGLM-6B/blob/main/resources/WECHAT.md" target="_blank">WeChat</a>\n</p>\n<p align="center">\n📍Experience the larger-scale ChatGLM model at <a href="https://www.chatglm.cn">chatglm.cn</a>\n</p>\n\n## 介绍\nChatGLM**2**-6B 是开源中英双语对话模

In [214]:
def get_model_readme(model_id):
    url = f"https://huggingface.co/{model_id}/resolve/main/README.md"
    response = requests.get(url)
    return response.text

In [215]:
model_readme = get_model_readme("THUDM/chatglm-6b-int4")

In [216]:
print(model_readme)

---
language:
- zh
- en
tags:
- glm
- chatglm
- thudm
---
# ChatGLM-6B-INT4
<p align="center">
    👋 Join our <a href="https://join.slack.com/t/chatglm/shared_invite/zt-1udqapmrr-ocT1DS_mxWe6dDY8ahRWzg" target="_blank">Slack</a> and <a href="https://github.com/THUDM/ChatGLM-6B/blob/main/resources/WECHAT.md" target="_blank">WeChat</a>
</p>

## 介绍
ChatGLM-6B 是一个开源的、支持中英双语问答的对话语言模型，基于 [General Language Model (GLM)](https://github.com/THUDM/GLM) 架构，具有 62 亿参数。结合模型量化技术，用户可以在消费级的显卡上进行本地部署（INT4 量化级别下最低只需 6GB 显存）。ChatGLM-6B 使用了和 [ChatGLM](https://chatglm.cn) 相同的技术，针对中文问答和对话进行了优化。经过约 1T 标识符的中英双语训练，辅以监督微调、反馈自助、人类反馈强化学习等技术的加持，62 亿参数的 ChatGLM-6B 已经能生成相当符合人类偏好的回答。

ChatGLM-6B-INT4 是 ChatGLM-6B 量化后的模型权重。具体的，ChatGLM-6B-INT4 对 ChatGLM-6B 中的 28 个 GLM Block 进行了 INT4 量化，没有对 Embedding 和 LM Head 进行量化。量化后的模型理论上 6G 显存（使用 CPU 即内存）即可推理，具有在嵌入式设备（如树莓派）上运行的可能。

在 CPU 上运行时，会根据硬件自动编译 CPU Kernel ，请确保已安装 GCC 和 OpenMP （Linux一般已安装，对于Windows则需手动安装），以获得最佳并行计算能力。

## 软件依赖

```shell
pip install protobuf transformers==4.27.1

### 7.3 基于HuggingFace网站搜索代码封装

In [224]:
def convert_keyword_huggingface(q):
    """
    将用户输入的问题转化为适合在huggingface上进行搜索的关键词
    """
    response = client.chat.completions.create(
            model="glm-4",
            messages=[
                 {"role": "system", "content": "你专门负责将用户的问题转化为huggingface上的搜索关键词，只返回一个你认为最合适的搜索关键词即可"},
                 {"role": "user", "content": "请问DeepSpeed是什么？"},
                 {"role": "assistant", "content": "DeepSpeed"},
                 {"role": "user", "content": q}
            ]
        )
    q = response.choices[0].message.content
    return q

In [229]:
def chat_with_huggingface(q):

    # 调用转化函数，将用户的问题转化为模型关键词
    query = convert_keyword_huggingface(q)
    
    search_results = search_models(query)

    sorted_results = sorted(search_results, key=lambda x: x['likes'], reverse=True)[:5]
    
    print('现有5个与“%s”相关的模型如下：' % query)
    for dic in sorted_results:
        print(dic['id'])


    #model_id = input('请输入想查询的模型ID：')
    model_id = 1

    model_readme = get_model_readme(model_id)

    messages=[
        {"role": "system", "content": model_readme},
        {"role": "user", "content": q}
        ]
    response = client.chat.completions.create(
                model="glm-4",
                messages=messages,) 
    print("智能搜索平台回答如下：")
    print(response.choices[0].message.content)
    
   


建议可以把整个流程整理成为一个多轮对话的机器人

In [230]:
chat_with_huggingface("介绍一下ChatGLM3")

现有5个与“ChatGLM3-6B”相关的模型如下：
THUDM/chatglm3-6b
THUDM/chatglm3-6b-32k
THUDM/chatglm3-6b-base
THUDM/chatglm3-6b-128k
shibing624/chatglm3-6b-csc-chinese-lora
智能搜索平台回答如下：
ChatGLM3 是由智谱AI和清华大学 KEG 实验室联合发布的新一代对话预训练模型。它是 ChatGLM 系列中的最新模型，继承了前两代模型在对话流畅性和部署门槛低等方面的优点，并引入了一些新的特性和改进。

以下是对 ChatGLM3 的一些关键介绍：

1. **更强大的基础模型**：ChatGLM3 的基础模型是 ChatGLM3-6B-Base，它使用了更多样化的训练数据、更充分的训练步数和更合理的训练策略。根据官方发布的信息，该模型在多个评测数据集上展现出了优异的性能，尤其在10B以下的基础模型中具有顶尖的性能。

2. **提升的性能**：通过多阶段增强预训练方法、丰富的训练数据和优化后的训练方案，ChatGLM3 相比前一代模型在性能上有了显著提升。在44个中英文公开数据集上的测试表明，ChatGLM3 在国内同尺寸模型中排名第一，尤其在数学、推理、代码和知识等方面表现突出。

3. **完整的功能支持**：ChatGLM3 采用了全新设计的 Prompt 格式，除了支持多轮对话外，还原生支持工具调用（Function Call）、代码执行（Code Interpreter）和 Agent 任务等复杂场景。

4. **全面的开源序列**：除了对话模型 ChatGLM3-6B，还包括基础模型 ChatGLM3-6B-Base 和长文本对话模型 ChatGLM3-6B-32K。这些模型的权重对学术研究完全开放，并在填写相应的问卷后允许免费商业使用。

5. **性能和功能层面的升级**：ChatGLM3-6B 在10B范围内的模型中性能最强，其推理能力甚至可以与 GPT-3.5 相媲美。此外，它还更新了多模态功能、代码解释器功能、联网功能以及 Agent 优化功能，这些核心功能的更新使其更加接近 GPT-4。

总体来说，ChatGLM3 作为一个开源的对话模型，不仅为学术研究提供了强大的工具，也为商业应用带来了更多的可能性。